In [1]:
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "1"
import sys
import torch
if torch.cuda.is_available():
    print(f"Physical GPU ID being used: 1")
    print(f"Logical GPU ID used by PyTorch: {torch.cuda.current_device()}")
    print(f"Device Name: {torch.cuda.get_device_name(0)}")
else:
    print("No CUDA device is available.")
from transformers.optimization import get_cosine_schedule_with_warmup
import torch.nn.functional as F
import torch_geometric.transforms as T
from ogb.graphproppred import PygGraphPropPredDataset, Evaluator
from torch_geometric.loader import DataLoader

import os
import random
import pandas as pd
import torch
import torch_geometric.transforms as T
from typing import Optional
import torch
from torch import Tensor
from torch_geometric.data import Data
from torch_geometric.data.datapipes import functional_transform
from torch_geometric.transforms import BaseTransform

import torch_geometric.transforms as T
from torch_geometric.datasets import WebKB
from torch_geometric.datasets import WebKB
from torch_geometric.datasets import Actor
from torch_geometric.datasets import GNNBenchmarkDataset
from torch_geometric.datasets import TUDataset
from sklearn.metrics import r2_score
from torch_geometric.data import DataLoader
from torch_geometric.datasets import MoleculeNet
from torch_geometric.nn import GCNConv
from torch_geometric.nn import global_mean_pool as gap, global_max_pool as gmp
from torch_geometric.utils import to_networkx
from torch.nn import Linear
from sklearn.model_selection import KFold
import matplotlib.pyplot as plt
import networkx as nx
import numpy as np
import os
import random
import pandas as pd

import time
import psutil
import torch
import torch.nn.functional as F
import warnings
warnings.filterwarnings("ignore")

from torch_geometric.utils.num_nodes import maybe_num_nodes
from torch_sparse import spspmm
from torch_sparse import coalesce
from torch_sparse import eye
from torch.nn import Parameter
from torch_scatter import scatter_add
from torch_scatter import scatter_max

from torch_scatter import scatter_add, scatter
from torch_geometric.nn.inits import uniform
from torch_geometric.nn.resolver import activation_resolver
from torch_geometric.nn import GCNConv, GATConv, LEConv, SAGEConv, GraphConv
from torch_geometric.nn import global_mean_pool as gap, global_max_pool as gmp

from dataclasses import dataclass
from typing import Optional

import torch
from torch import Tensor
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.nn import Parameter
from torch_geometric.nn.conv import MessagePassing
from torch_geometric.nn.conv.gcn_conv import gcn_norm
from torch_geometric.utils import add_remaining_self_loops, to_dense_adj, add_self_loops
from typing import Callable, Optional, Union
from torch_sparse import coalesce, transpose
from torch_scatter import scatter
from torch import Tensor

from typing import List, Optional, Tuple, Union
import math
import torch
from torch import Tensor
from torch_geometric.nn.models.mlp import Linear
from torch_geometric.nn.resolver import activation_resolver
from torch_geometric.nn import BatchNorm
import os.path as osp
import time
from math import ceil
from torch_geometric.datasets import TUDataset
import torch_geometric.transforms as T
from torch_geometric.data import DenseDataLoader
from torch_geometric.datasets import TUDataset
import torch_geometric.transforms as T
from torch_geometric.data import DenseDataLoader
import random
from torch_geometric.nn import GCNConv

import os.path as osp
import time
from math import ceil

import torch
import torch.nn.functional as F

import torch_geometric.transforms as T
from torch_geometric.datasets import TUDataset
from torch_geometric.loader import DenseDataLoader
from torch_geometric.nn import DenseGCNConv, dense_diff_pool
import torch
import torch.nn.functional as F

import torch_geometric.transforms as T
from torch_geometric.datasets import TUDataset
from torch_geometric.loader import DenseDataLoader
from torch_geometric.nn import DenseGCNConv, TopKPooling, SAGPooling
from torch_geometric.utils import to_dense_batch

from sklearn.metrics import f1_score

Physical GPU ID being used: 1
Logical GPU ID used by PyTorch: 0
Device Name: NVIDIA A100-SXM4-80GB


/home/boot/anaconda3/envs/ 1/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
import torch
from torch_geometric.datasets import WebKB
from torch_geometric.utils import to_networkx
import networkx as nx

data_path = "/data/ /Pooling/"
dataset_Texas = WebKB(root=data_path, name="Texas")
data = dataset_Texas[0]

G = to_networkx(data, to_undirected=True)

closeness_centrality = nx.closeness_centrality(G)

closeness_centrality_tensor = torch.tensor([closeness_centrality[i] for i in range(data.num_nodes)])

sorted_indices = torch.argsort(closeness_centrality_tensor)

num_nodes = data.num_nodes
train_size = int(0.5 * num_nodes)
val_size = int(0.25 * num_nodes)
test_size = num_nodes - train_size - val_size

train_indices = sorted_indices[:train_size]
val_indices = sorted_indices[train_size:train_size + val_size]
test_indices = sorted_indices[train_size + val_size:]

train_mask = torch.zeros(num_nodes, dtype=torch.bool)
val_mask = torch.zeros(num_nodes, dtype=torch.bool)
test_mask = torch.zeros(num_nodes, dtype=torch.bool)

train_mask[train_indices] = True
val_mask[val_indices] = True
test_mask[test_indices] = True

data.train_mask = train_mask
data.val_mask = val_mask
data.test_mask = test_mask

print(f"Train Mask: {train_mask.sum().item()} nodes")
print(f"Val Mask: {val_mask.sum().item()} nodes")
print(f"Test Mask: {test_mask.sum().item()} nodes")

train_closeness_avg = closeness_centrality_tensor[train_mask].float().mean().item()
val_closeness_avg = closeness_centrality_tensor[val_mask].float().mean().item()
test_closeness_avg = closeness_centrality_tensor[test_mask].float().mean().item()

print(f"Train Mask Average Closeness: {train_closeness_avg:.4f}")
print(f"Val Mask Average Closeness: {val_closeness_avg:.4f}")
print(f"Test Mask Average Closeness: {test_closeness_avg:.4f}")

Train Mask: 91 nodes
Val Mask: 45 nodes
Test Mask: 47 nodes
Train Mask Average Closeness: 0.2899
Val Mask Average Closeness: 0.3887
Test Mask Average Closeness: 0.4047


### TopKPooling

In [3]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
num_classes = dataset_Texas.num_classes
in_channels = dataset_Texas.num_features
hidden_channels = 64
out_channels = num_classes
depth = 2
pool_ratios = [0.7, 0.7]
class HierarchicalGCN_TOPK(torch.nn.Module):
    def __init__(self, in_channels, hidden_channels, out_channels, depth, pool_ratios, act=F.relu, sum_res=False):
        super(HierarchicalGCN_TOPK, self).__init__()
        assert depth >= 1
        self.in_channels = in_channels
        self.hidden_channels = hidden_channels
        self.out_channels = out_channels
        self.depth = depth
        self.pool_ratios = pool_ratios
        self.act = act
        self.sum_res = sum_res

        channels = self.hidden_channels

        self.down_convs = torch.nn.ModuleList()
        self.pools = torch.nn.ModuleList()
        self.down_convs.append(GCNConv(self.in_channels, channels))
        for i in range(self.depth):
            self.pools.append(TopKPooling(channels, ratio=pool_ratios[i]))
            self.down_convs.append(GCNConv(channels, channels))

        in_channels = channels if sum_res else 2 * channels

        self.up_convs = torch.nn.ModuleList()
        for i in range(self.depth):
            self.up_convs.append(GCNConv(in_channels, channels))
        self.up_convs.append(GCNConv(channels, self.out_channels))

    def forward(self, x, edge_index, batch=None):
        x, edge_index = x.to(device), edge_index.to(device)
        if batch is None:
            batch = edge_index.new_zeros(x.size(0))
        if batch is not None:
            batch = batch.to(device)

        x = F.dropout(x, p=0.5, training=self.training)
        x = self.down_convs[0](x, edge_index)
        x = self.act(x)

        xs = [x]
        edge_indices = [edge_index]
        perms = []

        for i in range(1, self.depth + 1):
            x, edge_index, _, batch, perm, _ = self.pools[i - 1](x, edge_index, batch=batch)
            x = self.down_convs[i](x, edge_index)
            x = self.act(x)

            if i < self.depth:
                xs.append(x)
                edge_indices.append(edge_index)
            perms.append(perm)

        for i in range(self.depth):
            j = self.depth - 1 - i

            res = xs[j]
            edge_index = edge_indices[j]
            perm = perms[j]

            up = torch.zeros_like(res)
            up[perm] = x
            x = res + up if self.sum_res else torch.cat((res, up), dim=-1)
            x = self.up_convs[i](x, edge_index)
            x = self.act(x)
        x = self.up_convs[-1](x, edge_index)

        return x

#  
def train_node_classifier(model, graph, optimizer, criterion, train_mask, val_mask, n_epochs=200, patience=150, min_delta=0.0001):
    best_val_acc = 0
    patience_counter = 0
    model.to(device)
    graph = graph.to(device)  

    for epoch in range(1, n_epochs + 1):
        model.train()
        optimizer.zero_grad()
        out = model(graph.x, graph.edge_index)
        loss = criterion(out[train_mask], graph.y[train_mask])
        loss.backward()
        optimizer.step()

        val_acc = eval_node_classifier(model, graph, val_mask)

        # Early stopping logic
        if val_acc > best_val_acc + min_delta:
            best_val_acc = val_acc
            patience_counter = 0  # Reset patience counter if improvement
        else:
            patience_counter += 1  # Increment patience counter if no improvement

        if patience_counter >= patience:
            # Early stopping
            break

    return model, best_val_acc

def eval_node_classifier(model, graph, mask):
    model.eval()
    pred = model(graph.x, graph.edge_index).argmax(dim=1)
    correct = (pred[mask] == graph.y[mask]).sum()
    acc = int(correct) / int(mask.sum())
    return acc

def compute_f1_scores(model, graph, mask):
    model.eval()
    pred = model(graph.x, graph.edge_index).argmax(dim=1)
    y_true = graph.y[mask].cpu().numpy()
    y_pred = pred[mask].cpu().numpy()
    micro_f1 = f1_score(y_true, y_pred, average='micro')
    macro_f1 = f1_score(y_true, y_pred, average='macro')
    return micro_f1, macro_f1

#  
seeds = [42, 123, 456]
micro_f1_scores = []
macro_f1_scores = []

for seed in seeds:
    torch.manual_seed(seed)
    np.random.seed(seed)

    model = HierarchicalGCN_TOPK(in_channels=in_channels, hidden_channels=hidden_channels,
                                 out_channels=out_channels, depth=depth, pool_ratios=pool_ratios)
    optimizer = torch.optim.Adam(model.parameters(), lr=0.001, weight_decay=5e-4)
    criterion = torch.nn.CrossEntropyLoss()

    #  
    model, best_val_acc = train_node_classifier(model, data, optimizer, criterion, data.train_mask, data.val_mask)

    #  
    micro_f1, macro_f1 = compute_f1_scores(model, data, data.val_mask)
    micro_f1_scores.append(micro_f1)
    macro_f1_scores.append(macro_f1)

#  
micro_f1_mean = np.mean(micro_f1_scores)
micro_f1_std = np.std(micro_f1_scores)
macro_f1_mean = np.mean(macro_f1_scores)
macro_f1_std = np.std(macro_f1_scores)

micro_f1_mean = np.mean(micro_f1_scores) * 100
micro_f1_std = np.std(micro_f1_scores) * 100
macro_f1_mean = np.mean(macro_f1_scores) * 100
macro_f1_std = np.std(macro_f1_scores) * 100

print(f"Micro F1: {micro_f1_mean:.2f}$\pm${micro_f1_std:.2f}")
print(f"Macro F1: {macro_f1_mean:.2f}$\pm${macro_f1_std:.2f}")

Micro F1: 61.48$\pm$1.05
Macro F1: 19.39$\pm$0.07


### SAGPooling

In [4]:
from torch_geometric.nn import SAGPooling
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
num_classes = dataset_Texas.num_classes
in_channels = dataset_Texas.num_features
hidden_channels = 64
out_channels = num_classes
depth = 2
pool_ratios = [0.7, 0.7]
class HierarchicalGCN_SAG(torch.nn.Module):
    def __init__(self, in_channels, hidden_channels, out_channels, depth, pool_ratios, act=F.relu, sum_res=False):
        super(HierarchicalGCN_SAG, self).__init__()
        assert depth >= 1
        self.in_channels = in_channels
        self.hidden_channels = hidden_channels
        self.out_channels = out_channels
        self.depth = depth
        self.pool_ratios = pool_ratios
        self.act = act
        self.sum_res = sum_res

        channels = self.hidden_channels

        self.down_convs = torch.nn.ModuleList()
        self.pools = torch.nn.ModuleList()
        self.down_convs.append(GCNConv(self.in_channels, channels))
        for i in range(self.depth):
            self.pools.append(SAGPooling(channels, ratio=pool_ratios[i]))
            self.down_convs.append(GCNConv(channels, channels))

        in_channels = channels if sum_res else 2 * channels

        self.up_convs = torch.nn.ModuleList()
        for i in range(self.depth):
            self.up_convs.append(GCNConv(in_channels, channels))
        self.up_convs.append(GCNConv(channels, self.out_channels))

    def forward(self, x, edge_index, batch=None):
        x, edge_index = x.to(device), edge_index.to(device)
        if batch is None:
            batch = edge_index.new_zeros(x.size(0))
        if batch is not None:
            batch = batch.to(device)

        x = F.dropout(x, p=0.5, training=self.training)
        x = self.down_convs[0](x, edge_index)
        x = self.act(x)

        xs = [x]
        edge_indices = [edge_index]
        perms = []

        for i in range(1, self.depth + 1):
            x, edge_index, _, batch, perm, _ = self.pools[i - 1](x, edge_index, None, batch)
            x = self.down_convs[i](x, edge_index)
            x = self.act(x)

            if i < self.depth:
                xs.append(x)
                edge_indices.append(edge_index)
            perms.append(perm)

        for i in range(self.depth):
            j = self.depth - 1 - i

            res = xs[j]
            edge_index = edge_indices[j]
            perm = perms[j]

            up = torch.zeros_like(res)
            up[perm] = x
            x = res + up if self.sum_res else torch.cat((res, up), dim=-1)
            x = self.up_convs[i](x, edge_index)
            x = self.act(x)
        x = self.up_convs[-1](x, edge_index)

        return x

#  
def train_node_classifier(model, graph, optimizer, criterion, train_mask, val_mask, n_epochs=200, patience=150, min_delta=0.0001):
    best_val_acc = 0
    patience_counter = 0
    model.to(device)
    graph = graph.to(device)  

    for epoch in range(1, n_epochs + 1):
        model.train()
        optimizer.zero_grad()
        out = model(graph.x, graph.edge_index)
        loss = criterion(out[train_mask], graph.y[train_mask])
        loss.backward()
        optimizer.step()

        val_acc = eval_node_classifier(model, graph, val_mask)

        # Early stopping logic
        if val_acc > best_val_acc + min_delta:
            best_val_acc = val_acc
            patience_counter = 0  # Reset patience counter if improvement
        else:
            patience_counter += 1  # Increment patience counter if no improvement

        if patience_counter >= patience:
            # Early stopping
            break

    return model, best_val_acc

def eval_node_classifier(model, graph, mask):
    model.eval()
    pred = model(graph.x, graph.edge_index).argmax(dim=1)
    correct = (pred[mask] == graph.y[mask]).sum()
    acc = int(correct) / int(mask.sum())
    return acc

def compute_f1_scores(model, graph, mask):
    model.eval()
    pred = model(graph.x, graph.edge_index).argmax(dim=1)
    y_true = graph.y[mask].cpu().numpy()
    y_pred = pred[mask].cpu().numpy()
    micro_f1 = f1_score(y_true, y_pred, average='micro')
    macro_f1 = f1_score(y_true, y_pred, average='macro')
    return micro_f1, macro_f1


seeds = [42, 123, 456]
micro_f1_scores = []
macro_f1_scores = []

for seed in seeds:
    torch.manual_seed(seed)
    np.random.seed(seed)

    model = HierarchicalGCN_SAG(in_channels=in_channels, hidden_channels=hidden_channels,
                                 out_channels=out_channels, depth=depth, pool_ratios=pool_ratios)
    optimizer = torch.optim.Adam(model.parameters(), lr=0.001, weight_decay=5e-4)
    criterion = torch.nn.CrossEntropyLoss()

    model, best_val_acc = train_node_classifier(model, data, optimizer, criterion, data.train_mask, data.val_mask)

    micro_f1, macro_f1 = compute_f1_scores(model, data, data.val_mask)
    micro_f1_scores.append(micro_f1)
    macro_f1_scores.append(macro_f1)

micro_f1_mean = np.mean(micro_f1_scores)
micro_f1_std = np.std(micro_f1_scores)
macro_f1_mean = np.mean(macro_f1_scores)
macro_f1_std = np.std(macro_f1_scores)

micro_f1_mean = np.mean(micro_f1_scores) * 100
micro_f1_std = np.std(micro_f1_scores) * 100
macro_f1_mean = np.mean(macro_f1_scores) * 100
macro_f1_std = np.std(macro_f1_scores) * 100

print(f"Micro F1: {micro_f1_mean:.2f}$\pm${micro_f1_std:.2f}")
print(f"Macro F1: {macro_f1_mean:.2f}$\pm${macro_f1_std:.2f}")

Micro F1: 62.22$\pm$1.81
Macro F1: 19.44$\pm$0.35


### ASAPooling

In [5]:
from torch_geometric.nn import ASAPooling
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
num_classes = dataset_Texas.num_classes
in_channels = dataset_Texas.num_features
hidden_channels = 64
out_channels = num_classes
depth = 2
pool_ratios = [0.7, 0.7]
class HierarchicalGCN_ASA(torch.nn.Module):
    def __init__(self, in_channels, hidden_channels, out_channels, depth, pool_ratios, act=F.relu, sum_res=False):
        super(HierarchicalGCN_ASA, self).__init__()
        assert depth >= 1
        self.in_channels = in_channels
        self.hidden_channels = hidden_channels
        self.out_channels = out_channels
        self.depth = depth
        self.pool_ratios = pool_ratios
        self.act = act
        self.sum_res = sum_res

        channels = self.hidden_channels

        self.down_convs = torch.nn.ModuleList()
        self.pools = torch.nn.ModuleList()
        self.down_convs.append(GCNConv(self.in_channels, channels))
        for i in range(self.depth):
            self.pools.append(ASAPooling(channels, ratio=pool_ratios[i]))
            self.down_convs.append(GCNConv(channels, channels))

        in_channels = channels if sum_res else 2 * channels

        self.up_convs = torch.nn.ModuleList()
        for i in range(self.depth):
            self.up_convs.append(GCNConv(in_channels, channels))
        self.up_convs.append(GCNConv(channels, self.out_channels))

    def forward(self, x, edge_index, batch=None):
        x, edge_index = x.to(device), edge_index.to(device)
        if batch is None:
            batch = edge_index.new_zeros(x.size(0))
        if batch is not None:
            batch = batch.to(device)

        x = F.dropout(x, p=0.5, training=self.training)
        x = self.down_convs[0](x, edge_index)
        x = self.act(x)

        xs = [x]
        edge_indices = [edge_index]
        perms = []

        for i in range(1, self.depth + 1):
            x, edge_index, _, batch, perm = self.pools[i - 1](x, edge_index, batch=batch)
            x = self.down_convs[i](x, edge_index)
            x = self.act(x)

            if i < self.depth:
                xs.append(x)
                edge_indices.append(edge_index)
            perms.append(perm)

        for i in range(self.depth):
            j = self.depth - 1 - i

            res = xs[j]
            edge_index = edge_indices[j]
            perm = perms[j]

            up = torch.zeros_like(res)
            up[perm] = x
            x = res + up if self.sum_res else torch.cat((res, up), dim=-1)
            x = self.up_convs[i](x, edge_index)
            x = self.act(x)
        x = self.up_convs[-1](x, edge_index)

        return x

def train_node_classifier(model, graph, optimizer, criterion, train_mask, val_mask, n_epochs=200, patience=150, min_delta=0.0001):
    best_val_acc = 0
    patience_counter = 0
    model.to(device)
    graph = graph.to(device)

    for epoch in range(1, n_epochs + 1):
        model.train()
        optimizer.zero_grad()
        out = model(graph.x, graph.edge_index)
        loss = criterion(out[train_mask], graph.y[train_mask])
        loss.backward()
        optimizer.step()

        # Evaluate on validation set
        val_acc = eval_node_classifier(model, graph, val_mask)

        # Early stopping logic
        if val_acc > best_val_acc + min_delta:
            best_val_acc = val_acc
            patience_counter = 0  # Reset patience counter if improvement
        else:
            patience_counter += 1  # Increment patience counter if no improvement

        if patience_counter >= patience:
            print(f'Early stopping at epoch {epoch}')
            break

        # Print progress
        print(f'Epoch {epoch:03d}: Loss = {loss.item():.4f}, Val Acc = {val_acc:.4f}')

    return model, best_val_acc


def eval_node_classifier(model, graph, mask):
    model.eval()
    pred = model(graph.x, graph.edge_index).argmax(dim=1)
    correct = (pred[mask] == graph.y[mask]).sum()
    acc = int(correct) / int(mask.sum())
    return acc

def compute_f1_scores(model, graph, mask):
    model.eval()
    pred = model(graph.x, graph.edge_index).argmax(dim=1)
    y_true = graph.y[mask].cpu().numpy()
    y_pred = pred[mask].cpu().numpy()
    micro_f1 = f1_score(y_true, y_pred, average='micro')
    macro_f1 = f1_score(y_true, y_pred, average='macro')
    return micro_f1, macro_f1


seeds = [42, 123, 456]
micro_f1_scores = []
macro_f1_scores = []

for seed in seeds:
    torch.manual_seed(seed)
    np.random.seed(seed)

    model = HierarchicalGCN_ASA(in_channels=in_channels, hidden_channels=hidden_channels,
                                 out_channels=out_channels, depth=depth, pool_ratios=pool_ratios)
    optimizer = torch.optim.Adam(model.parameters(), lr=0.001, weight_decay=5e-4)
    criterion = torch.nn.CrossEntropyLoss()

    model, best_val_acc = train_node_classifier(model, data, optimizer, criterion, data.train_mask, data.val_mask)

    micro_f1, macro_f1 = compute_f1_scores(model, data, data.val_mask)
    micro_f1_scores.append(micro_f1)
    macro_f1_scores.append(macro_f1)

micro_f1_mean = np.mean(micro_f1_scores)
micro_f1_std = np.std(micro_f1_scores)
macro_f1_mean = np.mean(macro_f1_scores)
macro_f1_std = np.std(macro_f1_scores)

micro_f1_mean = np.mean(micro_f1_scores) * 100
micro_f1_std = np.std(micro_f1_scores) * 100
macro_f1_mean = np.mean(macro_f1_scores) * 100
macro_f1_std = np.std(macro_f1_scores) * 100

print(f"Micro F1: {micro_f1_mean:.2f}$\pm${micro_f1_std:.2f}")
print(f"Macro F1: {macro_f1_mean:.2f}$\pm${macro_f1_std:.2f}")

Epoch 001: Loss = 1.6459, Val Acc = 0.7333
Epoch 002: Loss = 1.5798, Val Acc = 0.9111
Epoch 003: Loss = 1.5517, Val Acc = 0.9556
Epoch 004: Loss = 1.4947, Val Acc = 0.9111
Epoch 005: Loss = 1.4528, Val Acc = 0.8667
Epoch 006: Loss = 1.3967, Val Acc = 0.8000
Epoch 007: Loss = 1.3536, Val Acc = 0.7556
Epoch 008: Loss = 1.3126, Val Acc = 0.6889
Epoch 009: Loss = 1.2997, Val Acc = 0.7111
Epoch 010: Loss = 1.2761, Val Acc = 0.7333
Epoch 011: Loss = 1.2428, Val Acc = 0.7333
Epoch 012: Loss = 1.2028, Val Acc = 0.7333
Epoch 013: Loss = 1.1978, Val Acc = 0.7333
Epoch 014: Loss = 1.1691, Val Acc = 0.7333
Epoch 015: Loss = 1.1369, Val Acc = 0.7556
Epoch 016: Loss = 1.1157, Val Acc = 0.7556
Epoch 017: Loss = 1.1034, Val Acc = 0.7556
Epoch 018: Loss = 1.0756, Val Acc = 0.7556
Epoch 019: Loss = 1.0470, Val Acc = 0.7556
Epoch 020: Loss = 1.0535, Val Acc = 0.7556
Epoch 021: Loss = 0.9870, Val Acc = 0.7556
Epoch 022: Loss = 0.9725, Val Acc = 0.7333
Epoch 023: Loss = 0.9664, Val Acc = 0.7333
Epoch 024: 

### PANPooling

In [6]:
from torch_geometric.utils.num_nodes import maybe_num_nodes
from torch_sparse import spspmm
from torch_sparse import coalesce
from torch_sparse import eye
from torch.nn import Parameter
from torch_scatter import scatter_add
from torch_scatter import scatter_max

class PANPooling(torch.nn.Module):
    r""" General Graph pooling layer based on PAN, which can work with all layers.
    """
    def __init__(self, in_channels, ratio=0.5, pan_pool_weight=None, min_score=None, multiplier=1,
                 nonlinearity=torch.tanh, filter_size=3, panpool_filter_weight=None):
        super(PANPooling, self).__init__()

        self.in_channels = in_channels
        self.ratio = ratio
        self.min_score = min_score
        self.multiplier = multiplier
        self.nonlinearity = nonlinearity

        self.filter_size = filter_size
        if panpool_filter_weight is None:
            self.panpool_filter_weight = torch.nn.Parameter(0.5 * torch.ones(filter_size), requires_grad=True)

        self.transform = Parameter(torch.ones(in_channels), requires_grad=True)

        if pan_pool_weight is None:
            #self.weight = torch.tensor([0.7, 0.3], device=self.transform.device)
            self.pan_pool_weight = torch.nn.Parameter(0.5 * torch.ones(2), requires_grad=True)
        else:
            self.pan_pool_weight = pan_pool_weight

    def forward(self, x, edge_index, M=None, batch=None, num_nodes=None):

        """"""
        if batch is None:
            batch = edge_index.new_zeros(x.size(0))

        # Path integral
        num_nodes = maybe_num_nodes(edge_index, num_nodes)
        edge_index, edge_weight = self.panentropy_sparse(edge_index, num_nodes)

        # weighted degree
        num_nodes = x.size(0)
        degree = torch.zeros(num_nodes, device=edge_index.device)
        degree = scatter_add(edge_weight, edge_index[0], out=degree)

        # linear transform
        xtransform = torch.matmul(x, self.transform)

        # aggregate score
        x_transform_norm = xtransform #/ xtransform.norm(p=2, dim=-1)
        degree_norm = degree #/ degree.norm(p=2, dim=-1)
        score = self.pan_pool_weight[0] * x_transform_norm + self.pan_pool_weight[1] * degree_norm

        if self.min_score is None:
            score = self.nonlinearity(score)
        else:
            score = softmax(score, batch)

        perm = self.topk(score, self.ratio, batch, self.min_score)
        x = x[perm] * score[perm].view(-1, 1)
        x = self.multiplier * x if self.multiplier != 1 else x

        batch = batch[perm]
        edge_index, edge_weight = self.filter_adj(edge_index, edge_weight, perm, num_nodes=score.size(0))

        return x, edge_index, edge_weight, batch, perm, score[perm]

    def topk(self, x, ratio, batch, min_score=None, tol=1e-7):

        if min_score is not None:
            # Make sure that we do not drop all nodes in a graph.
            scores_max = scatter_max(x, batch)[0][batch] - tol
            scores_min = scores_max.clamp(max=min_score)

            perm = torch.nonzero(x > scores_min).view(-1)
        else:
            num_nodes = scatter_add(batch.new_ones(x.size(0)), batch, dim=0)
            batch_size, max_num_nodes = num_nodes.size(0), num_nodes.max().item()

            cum_num_nodes = torch.cat(
                [num_nodes.new_zeros(1),
                 num_nodes.cumsum(dim=0)[:-1]], dim=0)

            index = torch.arange(batch.size(0), dtype=torch.long, device=x.device)
            index = (index - cum_num_nodes[batch]) + (batch * max_num_nodes)

            dense_x = x.new_full((batch_size * max_num_nodes, ), -2)
            dense_x[index] = x
            dense_x = dense_x.view(batch_size, max_num_nodes)

            _, perm = dense_x.sort(dim=-1, descending=True)

            perm = perm + cum_num_nodes.view(-1, 1)
            perm = perm.view(-1)

            k = (ratio * num_nodes.to(torch.float)).ceil().to(torch.long)
            mask = [
                torch.arange(k[i], dtype=torch.long, device=x.device) +
                i * max_num_nodes for i in range(batch_size)
            ]
            mask = torch.cat(mask, dim=0)

            perm = perm[mask]

        return perm

    def filter_adj(self, edge_index, edge_weight, perm, num_nodes=None):

        num_nodes = maybe_num_nodes(edge_index, num_nodes)

        mask = perm.new_full((num_nodes, ), -1)
        i = torch.arange(perm.size(0), dtype=torch.long, device=perm.device)
        mask[perm] = i

        row, col = edge_index
        row, col = mask[row], mask[col]
        mask = (row >= 0) & (col >= 0)
        row, col = row[mask], col[mask]

        if edge_weight is not None:
            edge_weight = edge_weight[mask]

        return torch.stack([row, col], dim=0), edge_weight

    def panentropy_sparse(self, edge_index, num_nodes):

        edge_value = torch.ones(edge_index.size(1), device=edge_index.device)
        edge_index, edge_value = coalesce(edge_index, edge_value, num_nodes, num_nodes)

        # iteratively add weighted matrix power
        pan_index, pan_value = eye(num_nodes, device=edge_index.device)
        indextmp = pan_index.clone().to(edge_index.device)
        valuetmp = pan_value.clone().to(edge_index.device)

        pan_value = self.panpool_filter_weight[0] * pan_value

        for i in range(self.filter_size - 1):
            #indextmp, valuetmp = coalesce(indextmp, valuetmp, num_nodes, num_nodes)
            indextmp, valuetmp = spspmm(indextmp, valuetmp, edge_index, edge_value, num_nodes, num_nodes, num_nodes)
            valuetmp = valuetmp * self.panpool_filter_weight[i+1]
            indextmp, valuetmp = coalesce(indextmp, valuetmp, num_nodes, num_nodes)
            pan_index = torch.cat((pan_index, indextmp), 1)
            pan_value = torch.cat((pan_value, valuetmp))

        return coalesce(pan_index, pan_value, num_nodes, num_nodes, op='add')

In [7]:
from torch_geometric.nn import ASAPooling
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
num_classes = dataset_Texas.num_classes
in_channels = dataset_Texas.num_features
hidden_channels = 64
out_channels = num_classes
depth = 2
pool_ratios = [0.7, 0.7]
class HierarchicalGCN_PAN(torch.nn.Module):
    def __init__(self, in_channels, hidden_channels, out_channels, depth, pool_ratios, act=F.relu, sum_res=False):
        super(HierarchicalGCN_PAN, self).__init__()
        assert depth >= 1
        self.in_channels = in_channels
        self.hidden_channels = hidden_channels
        self.out_channels = out_channels
        self.depth = depth
        self.pool_ratios = pool_ratios
        self.act = act
        self.sum_res = sum_res

        channels = self.hidden_channels

        self.down_convs = torch.nn.ModuleList()
        self.pools = torch.nn.ModuleList()
        self.down_convs.append(GCNConv(self.in_channels, channels))
        for i in range(self.depth):
            self.pools.append(PANPooling(channels, ratio=pool_ratios[i]))
            self.down_convs.append(GCNConv(channels, channels))

        in_channels = channels if sum_res else 2 * channels

        self.up_convs = torch.nn.ModuleList()
        for i in range(self.depth):
            self.up_convs.append(GCNConv(in_channels, channels))
        self.up_convs.append(GCNConv(channels, self.out_channels))

    def forward(self, x, edge_index, batch=None):
        x, edge_index = x.to(device), edge_index.to(device)
        if batch is None:
            batch = edge_index.new_zeros(x.size(0))
        if batch is not None:
            batch = batch.to(device)

        x = F.dropout(x, p=0.5, training=self.training)
        x = self.down_convs[0](x, edge_index)
        x = self.act(x)

        xs = [x]
        edge_indices = [edge_index]
        perms = []

        for i in range(1, self.depth + 1):
            x, edge_index, _, batch, perm, score_perm = self.pools[i - 1](x, edge_index, batch=batch, M=None)
            x = self.down_convs[i](x, edge_index)
            x = self.act(x)

            if i < self.depth:
                xs.append(x)
                edge_indices.append(edge_index)
            perms.append(perm)

        for i in range(self.depth):
            j = self.depth - 1 - i

            res = xs[j]
            edge_index = edge_indices[j]
            perm = perms[j]

            up = torch.zeros_like(res)
            up[perm] = x
            x = res + up if self.sum_res else torch.cat((res, up), dim=-1)
            x = self.up_convs[i](x, edge_index)
            x = self.act(x)
        x = self.up_convs[-1](x, edge_index)

        return x

#  
def train_node_classifier(model, graph, optimizer, criterion, train_mask, val_mask, n_epochs=200, patience=150, min_delta=0.0001):
    best_val_acc = 0
    patience_counter = 0
    model.to(device)
    graph = graph.to(device)

    for epoch in range(1, n_epochs + 1):
        model.train()
        optimizer.zero_grad()
        out = model(graph.x, graph.edge_index)
        loss = criterion(out[train_mask], graph.y[train_mask])
        loss.backward()
        optimizer.step()

        # Evaluate on validation set
        val_acc = eval_node_classifier(model, graph, val_mask)

        # Early stopping logic
        if val_acc > best_val_acc + min_delta:
            best_val_acc = val_acc
            patience_counter = 0  # Reset patience counter if improvement
        else:
            patience_counter += 1  # Increment patience counter if no improvement

        if patience_counter >= patience:
            print(f'Early stopping at epoch {epoch}')
            break

        # Print progress
        print(f'Epoch {epoch:03d}: Loss = {loss.item():.4f}, Val Acc = {val_acc:.4f}')

    return model, best_val_acc


def eval_node_classifier(model, graph, mask):
    model.eval()
    pred = model(graph.x, graph.edge_index).argmax(dim=1)
    correct = (pred[mask] == graph.y[mask]).sum()
    acc = int(correct) / int(mask.sum())
    return acc

def compute_f1_scores(model, graph, mask):
    model.eval()
    pred = model(graph.x, graph.edge_index).argmax(dim=1)
    y_true = graph.y[mask].cpu().numpy()
    y_pred = pred[mask].cpu().numpy()
    micro_f1 = f1_score(y_true, y_pred, average='micro')
    macro_f1 = f1_score(y_true, y_pred, average='macro')
    return micro_f1, macro_f1


seeds = [42, 123, 456]
micro_f1_scores = []
macro_f1_scores = []

for seed in seeds:
    torch.manual_seed(seed)
    np.random.seed(seed)

    model = HierarchicalGCN_PAN(in_channels=in_channels, hidden_channels=hidden_channels,
                                 out_channels=out_channels, depth=depth, pool_ratios=pool_ratios)
    optimizer = torch.optim.Adam(model.parameters(), lr=0.001, weight_decay=5e-4)
    criterion = torch.nn.CrossEntropyLoss()

    model, best_val_acc = train_node_classifier(model, data, optimizer, criterion, data.train_mask, data.val_mask)

    micro_f1, macro_f1 = compute_f1_scores(model, data, data.val_mask)
    micro_f1_scores.append(micro_f1)
    macro_f1_scores.append(macro_f1)

micro_f1_mean = np.mean(micro_f1_scores)
micro_f1_std = np.std(micro_f1_scores)
macro_f1_mean = np.mean(macro_f1_scores)
macro_f1_std = np.std(macro_f1_scores)

micro_f1_mean = np.mean(micro_f1_scores) * 100
micro_f1_std = np.std(micro_f1_scores) * 100
macro_f1_mean = np.mean(macro_f1_scores) * 100
macro_f1_std = np.std(macro_f1_scores) * 100

print(f"Micro F1: {micro_f1_mean:.2f}$\pm${micro_f1_std:.2f}")
print(f"Macro F1: {macro_f1_mean:.2f}$\pm${macro_f1_std:.2f}")

Epoch 001: Loss = 1.5755, Val Acc = 0.0222
Epoch 002: Loss = 1.5313, Val Acc = 0.0222


Epoch 003: Loss = 1.4865, Val Acc = 0.0222
Epoch 004: Loss = 1.4344, Val Acc = 0.0444
Epoch 005: Loss = 1.4109, Val Acc = 0.2444
Epoch 006: Loss = 1.3382, Val Acc = 0.7111
Epoch 007: Loss = 1.3351, Val Acc = 0.8222
Epoch 008: Loss = 1.2861, Val Acc = 0.8222
Epoch 009: Loss = 1.2683, Val Acc = 0.8222
Epoch 010: Loss = 1.2319, Val Acc = 0.8222
Epoch 011: Loss = 1.1897, Val Acc = 0.8222
Epoch 012: Loss = 1.1717, Val Acc = 0.8222
Epoch 013: Loss = 1.1620, Val Acc = 0.8222
Epoch 014: Loss = 1.1288, Val Acc = 0.8222
Epoch 015: Loss = 1.1013, Val Acc = 0.8222
Epoch 016: Loss = 1.0606, Val Acc = 0.8000
Epoch 017: Loss = 1.0197, Val Acc = 0.7333
Epoch 018: Loss = 1.0707, Val Acc = 0.7333
Epoch 019: Loss = 0.9950, Val Acc = 0.7333
Epoch 020: Loss = 0.9810, Val Acc = 0.7333
Epoch 021: Loss = 0.9831, Val Acc = 0.7333
Epoch 022: Loss = 0.9480, Val Acc = 0.7333
Epoch 023: Loss = 0.9580, Val Acc = 0.7333
Epoch 024: Loss = 0.9210, Val Acc = 0.7333
Epoch 025: Loss = 0.8931, Val Acc = 0.7333
Epoch 026: 

### CoPooling

In [8]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.nn import Parameter
from torch_geometric.nn.conv import MessagePassing
from torch_geometric.nn.conv.gcn_conv import gcn_norm
from torch_geometric.utils import add_remaining_self_loops, to_dense_adj, add_self_loops
from typing import Callable, Optional, Union
from torch_sparse import coalesce, transpose
from torch_scatter import scatter

def cumsum(x: Tensor, dim: int = 0) -> Tensor:
    r"""Returns the cumulative sum of elements of :obj:`x`.
    In contrast to :meth:`torch.cumsum`, prepends the output with zero.

    Args:
        x (torch.Tensor): The input tensor.
        dim (int, optional): The dimension to do the operation over.
            (default: :obj:`0`)

    Example:
        >>> x = torch.tensor([2, 4, 1])
        >>> cumsum(x)
        tensor([0, 2, 6, 7])

    """
    size = x.size()[:dim] + (x.size(dim) + 1, ) + x.size()[dim + 1:]
    out = x.new_empty(size)

    out.narrow(dim, 0, 1).zero_()
    torch.cumsum(x, dim=dim, out=out.narrow(dim, 1, x.size(dim)))

    return out

def maybe_num_nodes(edge_index, num_nodes=None):
    if num_nodes is not None:
        return num_nodes
    elif isinstance(edge_index, Tensor):
        return int(edge_index.max()) + 1 if edge_index.numel() > 0 else 0
    else:
        return max(edge_index.size(0), edge_index.size(1))

def maybe_num_nodes(edge_index, num_nodes=None):
    if num_nodes is not None:
        return num_nodes
    elif isinstance(edge_index, Tensor):
        return int(edge_index.max()) + 1 if edge_index.numel() > 0 else 0
    else:
        return max(edge_index.size(0), edge_index.size(1))

def filter_adj(edge_index, edge_attr, perm, num_nodes=None):
    num_nodes = maybe_num_nodes(edge_index, num_nodes)

    mask = perm.new_full((num_nodes, ), -1)
    i = torch.arange(perm.size(0), dtype=torch.long, device=perm.device)
    mask[perm] = i

    row, col = edge_index
    row, col = mask[row], mask[col]
    mask = (row >= 0) & (col >= 0)
    row, col = row[mask], col[mask]

    if edge_attr is not None:
        edge_attr = edge_attr[mask]

    return torch.stack([row, col], dim=0), edge_attr

def topk(
    x: Tensor,
    ratio: Optional[Union[float, int]],
    batch: Tensor,
    min_score: Optional[float] = None,
    tol: float = 1e-7,
) -> Tensor:
    if min_score is not None:
        # Make sure that we do not drop all nodes in a graph.
        scores_max = scatter(x, batch, reduce='max')[batch] - tol
        scores_min = scores_max.clamp(max=min_score)

        perm = (x > scores_min).nonzero().view(-1)
        return perm

    if ratio is not None:
        num_nodes = scatter(batch.new_ones(x.size(0)), batch, reduce='sum')

        if ratio >= 1:
            k = num_nodes.new_full((num_nodes.size(0), ), int(ratio))
        else:
            k = (float(ratio) * num_nodes.to(x.dtype)).ceil().to(torch.long)

        x, x_perm = torch.sort(x.view(-1), descending=True)
        batch = batch[x_perm]
        batch, batch_perm = torch.sort(batch, descending=False, stable=True)

        arange = torch.arange(x.size(0), dtype=torch.long, device=x.device)
        ptr = cumsum(num_nodes)
        batched_arange = arange - ptr[batch]
        mask = batched_arange < k[batch]

        return x_perm[batch_perm[mask]]

    raise ValueError("At least one of the 'ratio' and 'min_score' parameters "
                     "must be specified")

class GPR_prop(MessagePassing):
    '''
    propagation class for GPR_GNN
    '''

    def __init__(self, K, alpha, Init, Gamma=None, bias=True, **kwargs):
        super(GPR_prop, self).__init__(aggr='add', **kwargs)
        self.K = K
        self.Init = Init
        self.alpha = alpha

        assert Init in ['SGC', 'PPR', 'NPPR', 'Random', 'WS']
        if Init == 'SGC':
            # SGC-like
            TEMP = 0.0*np.ones(K+1)
            TEMP[alpha] = 1.0
        elif Init == 'PPR':
            # PPR-like
            TEMP = alpha*(1-alpha)**np.arange(K+1)
            TEMP[-1] = (1-alpha)**K
        elif Init == 'NPPR':
            # Negative PPR
            TEMP = (alpha)**np.arange(K+1)
            TEMP = TEMP/np.sum(np.abs(TEMP))
        elif Init == 'Random':
            # Random
            bound = np.sqrt(3/(K+1))
            TEMP = np.random.uniform(-bound, bound, K+1)
            TEMP = TEMP/np.sum(np.abs(TEMP))
        elif Init == 'WS':
            # Specify Gamma
            TEMP = Gamma

        self.temp = Parameter(torch.tensor(TEMP))

    def reset_parameters(self):
        torch.nn.init.zeros_(self.temp)
        for k in range(self.K+1):
            self.temp.data[k] = self.alpha*(1-self.alpha)**k
        self.temp.data[-1] = (1-self.alpha)**self.K

    def forward(self, x, edge_index, edge_weight=None):
        edge_index, norm = gcn_norm(
            edge_index, edge_weight, num_nodes=x.size(0), dtype=x.dtype)

        hidden = x*(self.temp[0])
        for k in range(self.K):
            x = self.propagate(edge_index, x=x, norm=norm)
            gamma = self.temp[k+1]
            hidden = hidden + gamma*x
        return hidden

    def message(self, x_j, norm):
        return norm.view(-1, 1) * x_j

    def __repr__(self):
        return '{}(K={}, temp={})'.format(self.__class__.__name__, self.K,
                                           self.temp)


class NodeInformationScore(MessagePassing):
    def __init__(self, improved=False, cached=False, **kwargs):
        super(NodeInformationScore, self).__init__(aggr='add', **kwargs)

        self.improved = improved
        self.cached = cached
        self.cached_result = None
        self.cached_num_edges = None

    @staticmethod
    def norm(edge_index, num_nodes, edge_weight, dtype=None):
        if edge_weight is None:
            edge_weight = torch.ones((edge_index.size(1),), dtype=dtype, device=edge_index.device)

        edge_index, edge_weight = add_remaining_self_loops(edge_index, edge_weight, 0, num_nodes) # in case all the edges are removed

        edge_index = edge_index.type(torch.long)
        row, col = edge_index
        # print(row, col)
        # print(edge_weight.shape, row.shape, num_nodes)
        deg = scatter_add(edge_weight, row, dim=0, dim_size=num_nodes)
        deg_inv_sqrt = deg.pow(-0.5)
        deg_inv_sqrt[deg_inv_sqrt == float('inf')] = 0

        # row, col = edge_index
        expand_deg = torch.zeros((edge_weight.size(0),), dtype=dtype, device=edge_index.device)
        expand_deg[-num_nodes:] = torch.ones((num_nodes,), dtype=dtype, device=edge_index.device)

        return edge_index, expand_deg - deg_inv_sqrt[row] * edge_weight * deg_inv_sqrt[col]

    def forward(self, x, edge_index, edge_weight):
        if self.cached and self.cached_result is not None:
            if edge_index.size(1) != self.cached_num_edges:
                raise RuntimeError(
                    'Cached {} number of edges, but found {}'.format(self.cached_num_edges, edge_index.size(1)))

        if not self.cached or self.cached_result is None:
            self.cached_num_edges = edge_index.size(1)
            edge_index, norm = self.norm(edge_index, x.size(0), edge_weight, x.dtype)
            self.cached_result = edge_index, norm

        edge_index, norm = self.cached_result

        return self.propagate(edge_index, x=x, norm=norm)

    def message(self, x_j, norm):
        return norm.view(-1, 1) * x_j

    def update(self, aggr_out):
        return aggr_out

class graph_attention(torch.nn.Module):
    # reference: https://github.com/gordicaleksa/pytorch-GAT/blob/39c8f0ee634477033e8b1a6e9a6da3c7ed71bbd1/models/definitions/GAT.py#L324
    src_nodes_dim = 0  # position of source nodes in edge index
    trg_nodes_dim = 1  # position of target nodes in edge index

    nodes_dim = 0      # node dimension/axis
    head_dim = 1       # attention head dimension/axis

    def __init__(self, num_in_features, num_out_features, num_of_heads, dropout_prob=0.6, log_attention_weights=False):
        super().__init__()

        # Saving these as we'll need them in forward propagation in children layers (imp1/2/3)
        self.num_of_heads = num_of_heads
        self.num_out_features = num_out_features
        #
        # Trainable weights: linear projection matrix (denoted as "W" in the paper), attention target/source
        # (denoted as "a" in the paper) and bias (not mentioned in the paper but present in the official GAT repo)
        #

        # You can treat this one matrix as num_of_heads independent W matrices
        self.linear_proj = nn.Linear(num_in_features, num_of_heads * num_out_features, bias=False)

        # After we concatenate target node (node i) and source node (node j) we apply the additive scoring function
        # which gives us un-normalized score "e". Here we split the "a" vector - but the semantics remain the same.

        # Basically instead of doing [x, y] (concatenation, x/y are node feature vectors) and dot product with "a"
        # we instead do a dot product between x and "a_left" and y and "a_right" and we sum them up
        self.scoring_fn_target = nn.Parameter(torch.Tensor(1, num_of_heads, num_out_features))
        self.scoring_fn_source = nn.Parameter(torch.Tensor(1, num_of_heads, num_out_features))

        self.init_params()

    def init_params(self):
        """
        The reason we're using Glorot (aka Xavier uniform) initialization is because it's a default TF initialization:
            https://stackoverflow.com/questions/37350131/what-is-the-default-variable-initializer-in-tensorflow
        The original repo was developed in TensorFlow (TF) and they used the default initialization.
        Feel free to experiment - there may be better initializations depending on your problem.
        """
        nn.init.xavier_uniform_(self.linear_proj.weight)
        nn.init.xavier_uniform_(self.scoring_fn_target)
        nn.init.xavier_uniform_(self.scoring_fn_source)

    def forward(self, x, edge_index):
        #
        # Step 1: Linear Projection + regularization
        #

        in_nodes_features = x  # unpack data
        num_of_nodes = in_nodes_features.shape[self.nodes_dim]

        # shape = (N, FIN) * (FIN, NH*FOUT) -> (N, NH, FOUT) where NH - number of heads, FOUT - num of output features
        # We project the input node features into NH independent output features (one for each attention head)
        nodes_features_proj = self.linear_proj(in_nodes_features).view(-1, self.num_of_heads, self.num_out_features)

        #
        # Step 2: Edge attention calculation
        #

        # Apply the scoring function (* represents element-wise (a.k.a. Hadamard) product)
        # shape = (N, NH, FOUT) * (1, NH, FOUT) -> (N, NH, 1) -> (N, NH) because sum squeezes the last dimension
        # Optimization note: torch.sum() is as performant as .sum() in my experiments
        scores_source = (nodes_features_proj * self.scoring_fn_source).sum(dim=-1)
        scores_target = (nodes_features_proj * self.scoring_fn_target).sum(dim=-1)

        # We simply copy (lift) the scores for source/target nodes based on the edge index. Instead of preparing all
        # the possible combinations of scores we just prepare those that will actually be used and those are defined
        # by the edge index.
        # scores shape = (E, NH), nodes_features_proj_lifted shape = (E, NH, FOUT), E - number of edges in the graph
        scores_source_lifted, scores_target_lifted, nodes_features_proj_lifted = self.lift(scores_source, scores_target, nodes_features_proj, edge_index)
        scores_per_edge = scores_source_lifted + scores_target_lifted

        return torch.sigmoid(scores_per_edge)

    def lift(self, scores_source, scores_target, nodes_features_matrix_proj, edge_index):
        """
        Lifts i.e. duplicates certain vectors depending on the edge index.
        One of the tensor dims goes from N -> E (that's where the "lift" comes from).
        """
        src_nodes_index = edge_index[self.src_nodes_dim]
        trg_nodes_index = edge_index[self.trg_nodes_dim]

        # Using index_select is faster than "normal" indexing (scores_source[src_nodes_index]) in PyTorch!
        scores_source = scores_source.index_select(self.nodes_dim, src_nodes_index)
        scores_target = scores_target.index_select(self.nodes_dim, trg_nodes_index)
        nodes_features_matrix_proj_lifted = nodes_features_matrix_proj.index_select(self.nodes_dim, src_nodes_index)

        return scores_source, scores_target, nodes_features_matrix_proj_lifted



class CoPooling(torch.nn.Module):
    # reference for GAT code: https://github.com/PetarV-/GAT
    # reference for generalized pagerank code: https://github.com/jianhao2016/GPRGNN
    def __init__(self, ratio=0.5, K=0.05, edge_ratio=0.6, nhid=64, alpha=0.1, Init='Random', Gamma=None):
        super(CoPooling, self).__init__()
        self.ratio = ratio
        self.calc_information_score = NodeInformationScore()
        self.edge_ratio = edge_ratio

        self.prop1 = GPR_prop(K, alpha, Init, Gamma)

        score_dim = 32
        self.G_att = graph_attention(num_in_features=nhid, num_out_features=score_dim, num_of_heads=1)

        self.weight = Parameter(torch.Tensor(2*nhid, nhid))
        nn.init.xavier_uniform_(self.weight.data)
        self.bias = Parameter(torch.Tensor(nhid))
        nn.init.zeros_(self.bias.data)

        self.reset_parameters()

    def reset_parameters(self):
        nn.init.xavier_uniform_(self.weight.data)
        nn.init.zeros_(self.bias.data)
        self.prop1.reset_parameters()
        self.G_att.init_params()

    def forward(self, x, edge_index, edge_attr, batch=None, nodes_index=None, node_attr=None):
        if batch is None:
            batch = edge_index.new_zeros(x.size(0))
        ori_batch = batch.clone()
        device = x.device
        num_nodes = x.shape[0]

        # cut edges based on scores
        x_cut = self.prop1(x, edge_index) # run generalized pagerank to update features

        attention = self.G_att(x_cut, edge_index) # get the attention weights after sigmoid
        attention = attention.sum(dim=1) #sum the weights on head dim
        edge_index, attention = add_self_loops(edge_index, attention, 1.0, num_nodes) # add self loops in case no edges

        # to get a systemitic adj matrix
        edge_index_t, attention_t = transpose(edge_index, attention, num_nodes, num_nodes)
        edge_tmp = torch.cat((edge_index, edge_index_t), 1)
        att_tmp = torch.cat((attention, attention_t),0)
        edge_index, attention = coalesce(edge_tmp, att_tmp, num_nodes, num_nodes, 'mean')

        attention_np = attention.cpu().data.numpy()
        cut_val = np.percentile(attention_np, int(100*(1-self.edge_ratio))) # this is for keep the top edge_ratio edges
        attention = attention * (attention >= cut_val) # keep the edge_ratio higher weights of edges

        kep_idx = attention > 0.0
        cut_edge_index, cut_edge_attr = edge_index[:, kep_idx], attention[kep_idx]

        # Graph Pooling based on nodes
        x_information_score = self.calc_information_score(x, cut_edge_index, cut_edge_attr)
        score = torch.sum(torch.abs(x_information_score), dim=1)
        perm = topk(score, self.ratio, batch)
        x_topk = x[perm]
        batch = batch[perm]
        if nodes_index is not None:
            nodes_index = nodes_index[perm]

        if node_attr is not None:
            node_attr = node_attr[perm]
        if cut_edge_index is not None or cut_edge_index.nelement() != 0:
            induced_edge_index, induced_edge_attr = filter_adj(cut_edge_index, cut_edge_attr, perm, num_nodes=num_nodes)
        else:
            print('All edges are cut!')
            induced_edge_index, induced_edge_attr = cut_edge_index, cut_edge_attr

        # update node features
        attention_dense = (to_dense_adj(cut_edge_index, edge_attr=cut_edge_attr, max_num_nodes=num_nodes)).squeeze()
        x = F.relu(torch.matmul(torch.cat((x_topk, torch.matmul(attention_dense[perm],x)), 1), self.weight) + self.bias)

        return x, induced_edge_index, perm, induced_edge_attr, batch, nodes_index, node_attr, attention_dense

In [9]:
from torch_geometric.nn import ASAPooling
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
num_classes = dataset_Texas.num_classes
in_channels = dataset_Texas.num_features
hidden_channels = 64
out_channels = num_classes
depth = 2
pool_ratios = [0.7, 0.7]
class HierarchicalGCN_CO(torch.nn.Module):
    def __init__(self, in_channels, hidden_channels, out_channels, depth, pool_ratios, act=F.relu, sum_res=False):
        super(HierarchicalGCN_CO, self).__init__()
        assert depth >= 1
        self.in_channels = in_channels
        self.hidden_channels = hidden_channels
        self.out_channels = out_channels
        self.depth = depth
        self.pool_ratios = pool_ratios
        self.act = act
        self.sum_res = sum_res

        channels = self.hidden_channels

        self.down_convs = torch.nn.ModuleList()
        self.pools = torch.nn.ModuleList()
        self.down_convs.append(GCNConv(self.in_channels, channels))
        for i in range(self.depth):
            self.pools.append(CoPooling(ratio=pool_ratios[i], K=1, edge_ratio=0.6, nhid=64, alpha=0.1, Init='Random', Gamma=1.0))
            self.down_convs.append(GCNConv(channels, channels))

        in_channels = channels if sum_res else 2 * channels

        self.up_convs = torch.nn.ModuleList()
        for i in range(self.depth):
            self.up_convs.append(GCNConv(in_channels, channels))
        self.up_convs.append(GCNConv(channels, self.out_channels))

    def forward(self, x, edge_index, batch=None):
        x, edge_index = x.to(device), edge_index.to(device)
        if batch is None:
            batch = edge_index.new_zeros(x.size(0))
        if batch is not None:
            batch = batch.to(device)

        x = F.dropout(x, p=0.5, training=self.training)
        x = self.down_convs[0](x, edge_index)
        x = self.act(x)

        xs = [x]
        edge_indices = [edge_index]
        perms = []

        for i in range(1, self.depth + 1):
            x, edge_index, perm, _, batch, _, _, _ = self.pools[i - 1](x, edge_index, edge_attr=None, batch=batch)
            x = self.down_convs[i](x, edge_index)
            x = self.act(x)

            if i < self.depth:
                xs.append(x)
                edge_indices.append(edge_index)
            perms.append(perm)

        for i in range(self.depth):
            j = self.depth - 1 - i

            res = xs[j]
            edge_index = edge_indices[j]
            perm = perms[j]

            up = torch.zeros_like(res)
            up[perm] = x
            x = res + up if self.sum_res else torch.cat((res, up), dim=-1)
            x = self.up_convs[i](x, edge_index)
            x = self.act(x)
        x = self.up_convs[-1](x, edge_index)

        return x

#  
def train_node_classifier(model, graph, optimizer, criterion, train_mask, val_mask, n_epochs=200, patience=150, min_delta=0.0001):
    best_val_acc = 0
    patience_counter = 0
    model.to(device)
    graph = graph.to(device)  

    for epoch in range(1, n_epochs + 1):
        model.train()
        optimizer.zero_grad()
        out = model(graph.x, graph.edge_index)
        loss = criterion(out[train_mask], graph.y[train_mask])
        loss.backward()
        optimizer.step()

        val_acc = eval_node_classifier(model, graph, val_mask)

        # Early stopping logic
        if val_acc > best_val_acc + min_delta:
            best_val_acc = val_acc
            patience_counter = 0  # Reset patience counter if improvement
        else:
            patience_counter += 1  # Increment patience counter if no improvement

        if patience_counter >= patience:
            # Early stopping
            break

    return model, best_val_acc

def eval_node_classifier(model, graph, mask):
    model.eval()
    pred = model(graph.x, graph.edge_index).argmax(dim=1)
    correct = (pred[mask] == graph.y[mask]).sum()
    acc = int(correct) / int(mask.sum())
    return acc

def compute_f1_scores(model, graph, mask):
    model.eval()
    pred = model(graph.x, graph.edge_index).argmax(dim=1)
    y_true = graph.y[mask].cpu().numpy()
    y_pred = pred[mask].cpu().numpy()
    micro_f1 = f1_score(y_true, y_pred, average='micro')
    macro_f1 = f1_score(y_true, y_pred, average='macro')
    return micro_f1, macro_f1


seeds = [42, 123, 456]
micro_f1_scores = []
macro_f1_scores = []

for seed in seeds:
    torch.manual_seed(seed)
    np.random.seed(seed)

    model = HierarchicalGCN_CO(in_channels=in_channels, hidden_channels=hidden_channels,
                                 out_channels=out_channels, depth=depth, pool_ratios=pool_ratios)
    optimizer = torch.optim.Adam(model.parameters(), lr=0.001, weight_decay=5e-4)
    criterion = torch.nn.CrossEntropyLoss()

    model, best_val_acc = train_node_classifier(model, data, optimizer, criterion, data.train_mask, data.val_mask)

    micro_f1, macro_f1 = compute_f1_scores(model, data, data.val_mask)
    micro_f1_scores.append(micro_f1)
    macro_f1_scores.append(macro_f1)

micro_f1_mean = np.mean(micro_f1_scores)
micro_f1_std = np.std(micro_f1_scores)
macro_f1_mean = np.mean(macro_f1_scores)
macro_f1_std = np.std(macro_f1_scores)

micro_f1_mean = np.mean(micro_f1_scores) * 100
micro_f1_std = np.std(micro_f1_scores) * 100
macro_f1_mean = np.mean(macro_f1_scores) * 100
macro_f1_std = np.std(macro_f1_scores) * 100

print(f"Micro F1: {micro_f1_mean:.2f}$\pm${micro_f1_std:.2f}")
print(f"Macro F1: {macro_f1_mean:.2f}$\pm${macro_f1_std:.2f}")

Micro F1: 63.70$\pm$2.10
Macro F1: 19.72$\pm$0.39


### CGIPooling

In [10]:
from torch_scatter import scatter_add, scatter
from torch_geometric.nn.inits import uniform
from torch_geometric.nn.resolver import activation_resolver
from torch_geometric.nn import GCNConv, GATConv, LEConv, SAGEConv, GraphConv
from torch_geometric.nn import global_mean_pool as gap, global_max_pool as gmp

from dataclasses import dataclass
from typing import Optional

import torch
from torch import Tensor


@dataclass(init=False)
class SelectOutput:
    r"""The output of the :class:`Select` method, which holds an assignment
    from selected nodes to their respective cluster(s).

    Args:
        node_index (torch.Tensor): The indices of the selected nodes.
        num_nodes (int): The number of nodes.
        cluster_index (torch.Tensor): The indices of the clusters each node in
            :obj:`node_index` is assigned to.
        num_clusters (int): The number of clusters.
        weight (torch.Tensor, optional): A weight vector, denoting the strength
            of the assignment of a node to its cluster. (default: :obj:`None`)
    """
    node_index: Tensor
    num_nodes: int
    cluster_index: Tensor
    num_clusters: int
    weight: Optional[Tensor] = None

    def __init__(
        self,
        node_index: Tensor,
        num_nodes: int,
        cluster_index: Tensor,
        num_clusters: int,
        weight: Optional[Tensor] = None,
    ):
        if node_index.dim() != 1:
            raise ValueError(f"Expected 'node_index' to be one-dimensional "
                             f"(got {node_index.dim()} dimensions)")

        if cluster_index.dim() != 1:
            raise ValueError(f"Expected 'cluster_index' to be one-dimensional "
                             f"(got {cluster_index.dim()} dimensions)")

        if node_index.numel() != cluster_index.numel():
            raise ValueError(f"Expected 'node_index' and 'cluster_index' to "
                             f"hold the same number of values (got "
                             f"{node_index.numel()} and "
                             f"{cluster_index.numel()} values)")

        if weight is not None and weight.dim() != 1:
            raise ValueError(f"Expected 'weight' vector to be one-dimensional "
                             f"(got {weight.dim()} dimensions)")

        if weight is not None and weight.numel() != node_index.numel():
            raise ValueError(f"Expected 'weight' to hold {node_index.numel()} "
                             f"values (got {weight.numel()} values)")

        self.node_index = node_index
        self.num_nodes = num_nodes
        self.cluster_index = cluster_index
        self.num_clusters = num_clusters
        self.weight = weight




class Select(torch.nn.Module):
    r"""An abstract base class for implementing custom node selections as
    described in the `"Understanding Pooling in Graph Neural Networks"
    <https://arxiv.org/abs/1905.05178>`_ paper, which maps the nodes of an
    input graph to supernodes in the coarsened graph.

    Specifically, :class:`Select` returns a :class:`SelectOutput` output, which
    holds a (sparse) mapping :math:`\mathbf{C} \in {[0, 1]}^{N \times C}` that
    assigns selected nodes to one or more of :math:`C` super nodes.
    """
    def reset_parameters(self):
        r"""Resets all learnable parameters of the module."""
        pass

    def forward(self, *args, **kwargs) -> SelectOutput:
        raise NotImplementedError

    def __repr__(self) -> str:
        return f'{self.__class__.__name__}()'

def cumsum(x: Tensor, dim: int = 0) -> Tensor:
    r"""Returns the cumulative sum of elements of :obj:`x`.
    In contrast to :meth:`torch.cumsum`, prepends the output with zero.

    Args:
        x (torch.Tensor): The input tensor.
        dim (int, optional): The dimension to do the operation over.
            (default: :obj:`0`)

    Example:
        >>> x = torch.tensor([2, 4, 1])
        >>> cumsum(x)
        tensor([0, 2, 6, 7])

    """
    size = x.size()[:dim] + (x.size(dim) + 1, ) + x.size()[dim + 1:]
    out = x.new_empty(size)

    out.narrow(dim, 0, 1).zero_()
    torch.cumsum(x, dim=dim, out=out.narrow(dim, 1, x.size(dim)))

    return out

def maybe_num_nodes(edge_index, num_nodes=None):
    if num_nodes is not None:
        return num_nodes
    elif isinstance(edge_index, Tensor):
        return int(edge_index.max()) + 1 if edge_index.numel() > 0 else 0
    else:
        return max(edge_index.size(0), edge_index.size(1))

def maybe_num_nodes(edge_index, num_nodes=None):
    if num_nodes is not None:
        return num_nodes
    elif isinstance(edge_index, Tensor):
        return int(edge_index.max()) + 1 if edge_index.numel() > 0 else 0
    else:
        return max(edge_index.size(0), edge_index.size(1))

def filter_adj(edge_index, edge_attr, perm, num_nodes=None):
    num_nodes = maybe_num_nodes(edge_index, num_nodes)

    mask = perm.new_full((num_nodes, ), -1)
    i = torch.arange(perm.size(0), dtype=torch.long, device=perm.device)
    mask[perm] = i

    row, col = edge_index
    row, col = mask[row], mask[col]
    mask = (row >= 0) & (col >= 0)
    row, col = row[mask], col[mask]

    if edge_attr is not None:
        edge_attr = edge_attr[mask]

    return torch.stack([row, col], dim=0), edge_attr

def topk(
    x: Tensor,
    ratio: Optional[Union[float, int]],
    batch: Tensor,
    min_score: Optional[float] = None,
    tol: float = 1e-7,
) -> Tensor:
    if min_score is not None:
        # Make sure that we do not drop all nodes in a graph.
        scores_max = scatter(x, batch, reduce='max')[batch] - tol
        scores_min = scores_max.clamp(max=min_score)

        perm = (x > scores_min).nonzero().view(-1)
        return perm

    if ratio is not None:
        num_nodes = scatter(batch.new_ones(x.size(0)), batch, reduce='sum')

        if ratio >= 1:
            k = num_nodes.new_full((num_nodes.size(0), ), int(ratio))
        else:
            k = (float(ratio) * num_nodes.to(x.dtype)).ceil().to(torch.long)

        x, x_perm = torch.sort(x.view(-1), descending=True)
        batch = batch[x_perm]
        batch, batch_perm = torch.sort(batch, descending=False, stable=True)

        arange = torch.arange(x.size(0), dtype=torch.long, device=x.device)
        ptr = cumsum(num_nodes)
        batched_arange = arange - ptr[batch]
        mask = batched_arange < k[batch]

        return x_perm[batch_perm[mask]]

    raise ValueError("At least one of the 'ratio' and 'min_score' parameters "
                     "must be specified")

class Discriminator(torch.nn.Module):
    def __init__(self, in_channels):
        super(Discriminator, self).__init__()
        self.fc1 = nn.Linear(in_channels * 2, in_channels)
        self.fc2 = nn.Linear(in_channels, 1)

    def forward(self, x):
        x = F.leaky_relu(self.fc1(x), 0.2)
        x = F.sigmoid(self.fc2(x))
        return x


class CGIPool(torch.nn.Module):
    def __init__(self, in_channels, ratio=0.5, non_lin=torch.tanh):
        super(CGIPool, self).__init__()
        self.in_channels = in_channels
        self.ratio = ratio
        self.non_lin = non_lin
        self.hidden_dim = in_channels
        self.transform = GraphConv(in_channels, self.hidden_dim)
        self.pp_conv = GraphConv(self.hidden_dim, self.hidden_dim)
        self.np_conv = GraphConv(self.hidden_dim, self.hidden_dim)

        self.positive_pooling = GraphConv(self.hidden_dim, 1)
        self.negative_pooling = GraphConv(self.hidden_dim, 1)

        self.discriminator = Discriminator(self.hidden_dim)
        self.loss_fn = torch.nn.BCELoss()

    def forward(self, x, edge_index, edge_attr=None, batch=None):
        device = x.device  # 获取输入张量的设备信息

        if batch is None:
            batch = edge_index.new_zeros(x.size(0))

        x_transform = F.leaky_relu(self.transform(x, edge_index), 0.2)
        x_tp = F.leaky_relu(self.pp_conv(x, edge_index), 0.2)
        x_tn = F.leaky_relu(self.np_conv(x, edge_index), 0.2)
        s_pp = self.positive_pooling(x_tp, edge_index).squeeze()
        s_np = self.negative_pooling(x_tn, edge_index).squeeze()

        perm_positive = topk(s_pp, 1, batch)
        perm_negative = topk(s_np, 1, batch)
        x_pp = x_transform[perm_positive] * self.non_lin(s_pp[perm_positive]).view(-1, 1)
        x_np = x_transform[perm_negative] * self.non_lin(s_np[perm_negative]).view(-1, 1)

        x_pp_readout = gap(x_pp, batch[perm_positive])
        x_np_readout = gap(x_np, batch[perm_negative])
        x_readout = gap(x_transform, batch)

        positive_pair = torch.cat([x_pp_readout, x_readout], dim=1)
        negative_pair = torch.cat([x_np_readout, x_readout], dim=1)

        real = torch.ones(positive_pair.shape[0], device=device)  # 将张量移动到相应设备
        fake = torch.zeros(negative_pair.shape[0], device=device)  # 将张量移动到相应设备
        #real_loss = self.loss_fn(self.discriminator(positive_pair), real)
        #fake_loss = self.loss_fn(self.discriminator(negative_pair), fake)
        #discrimination_loss = (real_loss + fake_loss) / 2

        score = (s_pp - s_np)

        perm = topk(score, self.ratio, batch)
        x = x_transform[perm] * self.non_lin(score[perm]).view(-1, 1)
        batch = batch[perm]

        filter_edge_index, filter_edge_attr = filter_adj(edge_index, edge_attr, perm, num_nodes=score.size(0))
        return x, filter_edge_index, filter_edge_attr, batch, perm

In [11]:

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
num_classes = dataset_Texas.num_classes
in_channels = dataset_Texas.num_features
hidden_channels = 64
out_channels = num_classes
depth = 2
pool_ratios = [0.7, 0.7]
class HierarchicalGCN_CGI(torch.nn.Module):
    def __init__(self, in_channels, hidden_channels, out_channels, depth, pool_ratios, act=F.relu, sum_res=False):
        super(HierarchicalGCN_CGI, self).__init__()
        assert depth >= 1
        self.in_channels = in_channels
        self.hidden_channels = hidden_channels
        self.out_channels = out_channels
        self.depth = depth
        self.pool_ratios = pool_ratios
        self.act = act
        self.sum_res = sum_res

        channels = self.hidden_channels

        self.down_convs = torch.nn.ModuleList()
        self.pools = torch.nn.ModuleList()
        self.down_convs.append(GCNConv(self.in_channels, channels))
        for i in range(self.depth):
            self.pools.append(CGIPool(channels, ratio=pool_ratios[i]))
            self.down_convs.append(GCNConv(channels, channels))

        in_channels = channels if sum_res else 2 * channels

        self.up_convs = torch.nn.ModuleList()
        for i in range(self.depth):
            self.up_convs.append(GCNConv(in_channels, channels))
        self.up_convs.append(GCNConv(channels, self.out_channels))

    def forward(self, x, edge_index, batch=None):
        x, edge_index = x.to(device), edge_index.to(device)
        if batch is None:
            batch = edge_index.new_zeros(x.size(0))
        if batch is not None:
            batch = batch.to(device)

        x = F.dropout(x, p=0.5, training=self.training)
        x = self.down_convs[0](x, edge_index)
        x = self.act(x)

        xs = [x]
        edge_indices = [edge_index]
        perms = []

        for i in range(1, self.depth + 1):
            x, edge_index, _, batch, perm = self.pools[i - 1](x, edge_index, None, batch)
            x = self.down_convs[i](x, edge_index)
            x = self.act(x)

            if i < self.depth:
                xs.append(x)
                edge_indices.append(edge_index)
            perms.append(perm)

        for i in range(self.depth):
            j = self.depth - 1 - i

            res = xs[j]
            edge_index = edge_indices[j]
            perm = perms[j]

            up = torch.zeros_like(res)
            up[perm] = x
            x = res + up if self.sum_res else torch.cat((res, up), dim=-1)
            x = self.up_convs[i](x, edge_index)
            x = self.act(x)
        x = self.up_convs[-1](x, edge_index)

        return x

#  
def train_node_classifier(model, graph, optimizer, criterion, train_mask, val_mask, n_epochs=200, patience=150, min_delta=0.0001):
    best_val_acc = 0
    patience_counter = 0
    model.to(device)
    graph = graph.to(device)  

    for epoch in range(1, n_epochs + 1):
        model.train()
        optimizer.zero_grad()
        out = model(graph.x, graph.edge_index)
        loss = criterion(out[train_mask], graph.y[train_mask])
        loss.backward()
        optimizer.step()

        val_acc = eval_node_classifier(model, graph, val_mask)

        # Early stopping logic
        if val_acc > best_val_acc + min_delta:
            best_val_acc = val_acc
            patience_counter = 0  # Reset patience counter if improvement
        else:
            patience_counter += 1  # Increment patience counter if no improvement

        if patience_counter >= patience:
            # Early stopping
            break

    return model, best_val_acc

def eval_node_classifier(model, graph, mask):
    model.eval()
    pred = model(graph.x, graph.edge_index).argmax(dim=1)
    correct = (pred[mask] == graph.y[mask]).sum()
    acc = int(correct) / int(mask.sum())
    return acc

def compute_f1_scores(model, graph, mask):
    model.eval()
    pred = model(graph.x, graph.edge_index).argmax(dim=1)
    y_true = graph.y[mask].cpu().numpy()
    y_pred = pred[mask].cpu().numpy()
    micro_f1 = f1_score(y_true, y_pred, average='micro')
    macro_f1 = f1_score(y_true, y_pred, average='macro')
    return micro_f1, macro_f1


seeds = [42, 123, 456]
micro_f1_scores = []
macro_f1_scores = []

for seed in seeds:
    torch.manual_seed(seed)
    np.random.seed(seed)

    model = HierarchicalGCN_CGI(in_channels=in_channels, hidden_channels=hidden_channels,
                                 out_channels=out_channels, depth=depth, pool_ratios=pool_ratios)
    optimizer = torch.optim.Adam(model.parameters(), lr=0.001, weight_decay=5e-4)
    criterion = torch.nn.CrossEntropyLoss()

    model, best_val_acc = train_node_classifier(model, data, optimizer, criterion, data.train_mask, data.val_mask)

    micro_f1, macro_f1 = compute_f1_scores(model, data, data.val_mask)
    micro_f1_scores.append(micro_f1)
    macro_f1_scores.append(macro_f1)

micro_f1_mean = np.mean(micro_f1_scores)
micro_f1_std = np.std(micro_f1_scores)
macro_f1_mean = np.mean(macro_f1_scores)
macro_f1_std = np.std(macro_f1_scores)

micro_f1_mean = np.mean(micro_f1_scores) * 100
micro_f1_std = np.std(micro_f1_scores) * 100
macro_f1_mean = np.mean(macro_f1_scores) * 100
macro_f1_std = np.std(macro_f1_scores) * 100

print(f"Micro F1: {micro_f1_mean:.2f}$\pm${micro_f1_std:.2f}")
print(f"Macro F1: {macro_f1_mean:.2f}$\pm${macro_f1_std:.2f}")

Micro F1: 60.74$\pm$1.05
Macro F1: 19.25$\pm$0.18


### KMISPooling

In [12]:
from typing import Callable, Optional, Tuple, Union
from torch_geometric.typing import Adj, OptTensor, PairTensor, Tensor
Scorer = Callable[[Tensor, Adj, OptTensor, OptTensor], Tensor]
from torch_sparse import SparseTensor, remove_diag
from torch_geometric.nn.aggr import Aggregation
from torch_geometric.nn.dense import Linear
from torch.nn import Module
from torch_scatter import scatter_max, scatter_min

def maximal_independent_set(edge_index: Adj, k: int = 1,
                            perm: OptTensor = None) -> Tensor:
    r"""Returns a Maximal :math:`k`-Independent Set of a graph, i.e., a set of
    nodes (as a :class:`ByteTensor`) such that none of them are :math:`k`-hop
    neighbors, and any node in the graph has a :math:`k`-hop neighbor in the
    returned set.
    The algorithm greedily selects the nodes in their canonical order. If a
    permutation :obj:`perm` is provided, the nodes are extracted following
    that permutation instead.
    This method follows `Blelloch's Alogirithm
    <https://arxiv.org/abs/1202.3205>`_ for :math:`k = 1`, and its
    generalization by `Bacciu et al. <https://arxiv.org/abs/2208.03523>`_ for
    higher values of :math:`k`.
    Args:
        edge_index (Tensor or SparseTensor): The graph connectivity.
        k (int): The :math:`k` value (defaults to 1).
        perm (LongTensor, optional): Permutation vector. Must be of size
            :obj:`(n,)` (defaults to :obj:`None`).
    :rtype: :class:`ByteTensor`
    """
    if isinstance(edge_index, SparseTensor):
        row, col, _ = edge_index.coo()
        device = edge_index.device()
        n = edge_index.size(0)
    else:
        row, col = edge_index[0], edge_index[1]
        device = row.device
        n = edge_index.max().item() + 1

    if perm is None:
        rank = torch.arange(n, dtype=torch.long, device=device)
    else:
        rank = torch.zeros_like(perm)
        rank[perm] = torch.arange(n, dtype=torch.long, device=device)

    mis = torch.zeros(n, dtype=torch.bool, device=device)
    mask = mis.clone()
    min_rank = rank.clone()

    while not mask.all():
        for _ in range(k):
            min_neigh = torch.full_like(min_rank, fill_value=n)
            scatter_min(min_rank[row], col, out=min_neigh)
            torch.minimum(min_neigh, min_rank, out=min_rank)  # self-loops

        mis = mis | torch.eq(rank, min_rank)
        mask = mis.clone().byte()

        for _ in range(k):
            max_neigh = torch.full_like(mask, fill_value=0)
            scatter_max(mask[row], col, out=max_neigh)
            torch.maximum(max_neigh, mask, out=mask)  # self-loops

        mask = mask.to(dtype=torch.bool)
        min_rank = rank.clone()
        min_rank[mask] = n

    return mis

def maximal_independent_set_cluster(edge_index: Adj, k: int = 1,
                                    perm: OptTensor = None) -> PairTensor:
    r"""Computes the Maximal :math:`k`-Independent Set (:math:`k`-MIS)
    clustering of a graph, as defined in `"Generalizing Downsampling from
    Regular Data to Graphs" <https://arxiv.org/abs/2208.03523>`_.
    The algorithm greedily selects the nodes in their canonical order. If a
    permutation :obj:`perm` is provided, the nodes are extracted following
    that permutation instead.
    This method returns both the :math:`k`-MIS and the clustering, where the
    :math:`c`-th cluster refers to the :math:`c`-th element of the
    :math:`k`-MIS.
    Args:
        edge_index (Tensor or SparseTensor): The graph connectivity.
        k (int): The :math:`k` value (defaults to 1).
        perm (LongTensor, optional): Permutation vector. Must be of size
            :obj:`(n,)` (defaults to :obj:`None`).
    :rtype: (:class:`ByteTensor`, :class:`LongTensor`)
    """
    mis = maximal_independent_set(edge_index=edge_index, k=k, perm=perm)
    n, device = mis.size(0), mis.device

    if isinstance(edge_index, SparseTensor):
        row, col, _ = edge_index.coo()
    else:
        row, col = edge_index[0], edge_index[1]

    if perm is None:
        rank = torch.arange(n, dtype=torch.long, device=device)
    else:
        rank = torch.zeros_like(perm)
        rank[perm] = torch.arange(n, dtype=torch.long, device=device)

    min_rank = torch.full((n, ), fill_value=n, dtype=torch.long, device=device)
    rank_mis = rank[mis]
    min_rank[mis] = rank_mis

    for _ in range(k):
        min_neigh = torch.full_like(min_rank, fill_value=n)
        scatter_min(min_rank[row], col, out=min_neigh)
        torch.minimum(min_neigh, min_rank, out=min_rank)

    _, clusters = torch.unique(min_rank, return_inverse=True)
    perm = torch.argsort(rank_mis)
    return mis, perm[clusters]


class KMISPooling(Module):

    _heuristics = {None, 'greedy', 'w-greedy'}
    _passthroughs = {None, 'before', 'after'}
    _scorers = {
        'linear',
        'random',
        'constant',
        'canonical',
        'first',
        'last',
    }

    def __init__(self, in_channels: Optional[int] = None, k: int = 1,
                 scorer: Union[Scorer, str] = 'linear',
                 score_heuristic: Optional[str] = 'greedy',
                 score_passthrough: Optional[str] = 'before',
                 aggr_x: Optional[Union[str, Aggregation]] = None,
                 aggr_edge: str = 'sum',
                 aggr_score: Callable[[Tensor, Tensor], Tensor] = torch.mul,
                 remove_self_loops: bool = True) -> None:
        super(KMISPooling, self).__init__()
        assert score_heuristic in self._heuristics, \
            "Unrecognized `score_heuristic` value."
        assert score_passthrough in self._passthroughs, \
            "Unrecognized `score_passthrough` value."

        if not callable(scorer):
            assert scorer in self._scorers, \
                "Unrecognized `scorer` value."

        self.k = k
        self.scorer = scorer
        self.score_heuristic = score_heuristic
        self.score_passthrough = score_passthrough

        self.aggr_x = aggr_x
        self.aggr_edge = aggr_edge
        self.aggr_score = aggr_score
        self.remove_self_loops = remove_self_loops

        if scorer == 'linear':
            assert self.score_passthrough is not None, \
                "`'score_passthrough'` must not be `None`" \
                " when using `'linear'` scorer"

            self.lin = nn.Linear(in_channels, 1)

    def _apply_heuristic(self, x: Tensor, adj: SparseTensor) -> Tensor:
        if self.score_heuristic is None:
            return x

        row, col, _ = adj.coo()
        x = x.view(-1)

        if self.score_heuristic == 'greedy':
            k_sums = torch.ones_like(x)
        else:
            k_sums = x.clone()

        for _ in range(self.k):
            scatter_add(k_sums[row], col, out=k_sums)

        return x / k_sums

    def _scorer(self, x: Tensor, edge_index: Adj, edge_attr: OptTensor = None,
                batch: OptTensor = None) -> Tensor:
        if self.scorer == 'linear':
            return self.lin(x).sigmoid()

        if self.scorer == 'random':
            return torch.rand((x.size(0), 1), device=x.device)

        if self.scorer == 'constant':
            return torch.ones((x.size(0), 1), device=x.device)

        if self.scorer == 'canonical':
            return -torch.arange(x.size(0), device=x.device).view(-1, 1)

        if self.scorer == 'first':
            return x[..., [0]]

        if self.scorer == 'last':
            return x[..., [-1]]

        return self.scorer(x, edge_index, edge_attr, batch)


    def forward(self, x: Tensor, edge_index: Adj,
                edge_attr: OptTensor = None,
                batch: OptTensor = None) \
            -> Tuple[Tensor, Adj, OptTensor, OptTensor, Tensor, Tensor]:
        """"""
        edge_index = edge_index.long()
        adj, n = edge_index, x.size(0)

        if not isinstance(edge_index, SparseTensor):
            adj = SparseTensor.from_edge_index(edge_index, edge_attr, (n, n))

        score = self._scorer(x, edge_index, edge_attr, batch)
        updated_score = self._apply_heuristic(score, adj)
        perm = torch.argsort(updated_score.view(-1), 0, descending=True)

        mis, cluster = maximal_independent_set_cluster(adj, self.k, perm)

        row, col, val = adj.coo()
        c = mis.sum()

        if val is None:
            val = torch.ones_like(row, dtype=torch.float)

        adj = SparseTensor(row=cluster[row], col=cluster[col], value=val,
                           is_sorted=False,
                           sparse_sizes=(c, c)).coalesce(self.aggr_edge)

        if self.remove_self_loops:
            adj = remove_diag(adj)

        if self.score_passthrough == 'before':
            x = self.aggr_score(x, score)

        if self.aggr_x is None:
            x = x[mis]
        elif isinstance(self.aggr_x, str):
            x = scatter(x, cluster, dim=0, dim_size=mis.sum(),
                        reduce=self.aggr_x)
        else:
            x = self.aggr_x(x, cluster, dim_size=c)

        if self.score_passthrough == 'after':
            x = self.aggr_score(x, score[mis])

        if isinstance(edge_index, SparseTensor):
            edge_index, edge_attr = adj, None

        else:
            row, col, edge_attr = adj.coo()
            edge_index = torch.stack([row, col])

        if batch is not None:
            batch = batch[mis]

        #attn = x
        #select_out = topk(attn, batch)
        perm = perm[mis]


        return x, edge_index, edge_attr, batch, mis, cluster, perm

    def __repr__(self):
        if self.scorer == 'linear':
            channels = f"in_channels={self.lin.in_channels}, "
        else:
            channels = ""

        return f'{self.__class__.__name__}({channels}k={self.k})'

In [13]:

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
num_classes = dataset_Texas.num_classes
in_channels = dataset_Texas.num_features
hidden_channels = 64
out_channels = num_classes
depth = 2

class HierarchicalGCN_KMIS(torch.nn.Module):
    def __init__(self, in_channels, hidden_channels, out_channels, depth, pool_ratios, act=F.relu, sum_res=False):
        super(HierarchicalGCN_KMIS, self).__init__()
        assert depth >= 1
        self.in_channels = in_channels
        self.hidden_channels = hidden_channels
        self.out_channels = out_channels
        self.depth = depth
        self.pool_ratios = pool_ratios
        self.act = act
        self.sum_res = sum_res

        channels = self.hidden_channels

        self.down_convs = torch.nn.ModuleList()
        self.pools = torch.nn.ModuleList()
        self.down_convs.append(GCNConv(self.in_channels, channels))
        for i in range(self.depth):
            self.pools.append(KMISPooling(64, k=1, aggr_x='sum'))
            self.down_convs.append(GCNConv(channels, channels))

        in_channels = channels if sum_res else 2 * channels

        self.up_convs = torch.nn.ModuleList()
        for i in range(self.depth):
            self.up_convs.append(GCNConv(in_channels, channels))
        self.up_convs.append(GCNConv(channels, self.out_channels))

    def forward(self, x, edge_index, batch=None):
        x, edge_index = x.to(device), edge_index.to(device)
        if batch is None:
            batch = edge_index.new_zeros(x.size(0))
        if batch is not None:
            batch = batch.to(device)

        x = F.dropout(x, p=0.5, training=self.training)
        x = self.down_convs[0](x, edge_index)
        x = self.act(x)

        xs = [x]
        edge_indices = [edge_index]
        perms = []

        for i in range(1, self.depth + 1):
            x, edge_index, _, batch, _, cluster, perm = self.pools[i - 1](x, edge_index, batch=batch)
            x = self.down_convs[i](x, edge_index)
            x = self.act(x)

            if i < self.depth:
                xs.append(x)
                edge_indices.append(edge_index)
            perms.append(perm)

        for i in range(self.depth):
            j = self.depth - 1 - i

            res = xs[j]
            edge_index = edge_indices[j]
            perm = perms[j]

            up = torch.zeros_like(res)
            up[perm] = x
            x = res + up if self.sum_res else torch.cat((res, up), dim=-1)
            x = self.up_convs[i](x, edge_index)
            x = self.act(x)
        x = self.up_convs[-1](x, edge_index)

        return x

#  
def train_node_classifier(model, graph, optimizer, criterion, train_mask, val_mask, n_epochs=200, patience=150, min_delta=0.0001):
    best_val_acc = 0
    patience_counter = 0
    model.to(device)
    graph = graph.to(device)  

    for epoch in range(1, n_epochs + 1):
        model.train()
        optimizer.zero_grad()
        out = model(graph.x, graph.edge_index)
        loss = criterion(out[train_mask], graph.y[train_mask])
        loss.backward()
        optimizer.step()

        val_acc = eval_node_classifier(model, graph, val_mask)

        # Early stopping logic
        if val_acc > best_val_acc + min_delta:
            best_val_acc = val_acc
            patience_counter = 0  # Reset patience counter if improvement
        else:
            patience_counter += 1  # Increment patience counter if no improvement

        if patience_counter >= patience:
            # Early stopping
            break

    return model, best_val_acc

def eval_node_classifier(model, graph, mask):
    model.eval()
    pred = model(graph.x, graph.edge_index).argmax(dim=1)
    correct = (pred[mask] == graph.y[mask]).sum()
    acc = int(correct) / int(mask.sum())
    return acc

def compute_f1_scores(model, graph, mask):
    model.eval()
    pred = model(graph.x, graph.edge_index).argmax(dim=1)
    y_true = graph.y[mask].cpu().numpy()
    y_pred = pred[mask].cpu().numpy()
    micro_f1 = f1_score(y_true, y_pred, average='micro')
    macro_f1 = f1_score(y_true, y_pred, average='macro')
    return micro_f1, macro_f1


seeds = [42, 123, 456]
micro_f1_scores = []
macro_f1_scores = []

for seed in seeds:
    torch.manual_seed(seed)
    np.random.seed(seed)

    model = HierarchicalGCN_KMIS(in_channels=in_channels, hidden_channels=hidden_channels,
                                 out_channels=out_channels, depth=depth, pool_ratios=pool_ratios)
    optimizer = torch.optim.Adam(model.parameters(), lr=0.001, weight_decay=5e-4)
    criterion = torch.nn.CrossEntropyLoss()

    model, best_val_acc = train_node_classifier(model, data, optimizer, criterion, data.train_mask, data.val_mask)

    micro_f1, macro_f1 = compute_f1_scores(model, data, data.val_mask)
    micro_f1_scores.append(micro_f1)
    macro_f1_scores.append(macro_f1)

micro_f1_mean = np.mean(micro_f1_scores)
micro_f1_std = np.std(micro_f1_scores)
macro_f1_mean = np.mean(macro_f1_scores)
macro_f1_std = np.std(macro_f1_scores)

micro_f1_mean = np.mean(micro_f1_scores) * 100
micro_f1_std = np.std(micro_f1_scores) * 100
macro_f1_mean = np.mean(macro_f1_scores) * 100
macro_f1_std = np.std(macro_f1_scores) * 100

print(f"Micro F1: {micro_f1_mean:.2f}$\pm${micro_f1_std:.2f}")
print(f"Macro F1: {macro_f1_mean:.2f}$\pm${macro_f1_std:.2f}")

Micro F1: 63.70$\pm$1.05
Macro F1: 19.72$\pm$0.20


### GSAPooling

In [14]:
import math
from typing import Union, Optional, Callable
from torch_scatter import scatter_add, scatter_max
from torch_geometric.utils import softmax
from torch_geometric.nn import GCNConv, SAGEConv, GATConv, ChebConv, GraphConv

def uniform(size, tensor):
    if tensor is not None:
        bound = 1.0 / math.sqrt(size)
        tensor.data.uniform_(-bound, bound)


def maybe_num_nodes(edge_index, num_nodes=None):
    if num_nodes is not None:
        return num_nodes
    elif isinstance(edge_index, Tensor):
        return int(edge_index.max()) + 1 if edge_index.numel() > 0 else 0
    else:
        return max(edge_index.size(0), edge_index.size(1))


def topk(x, ratio, batch, min_score=None, tol=1e-7):
    if min_score is not None:
        # Make sure that we do not drop all nodes in a graph.
        scores_max = scatter_max(x, batch)[0][batch] - tol
        scores_min = scores_max.clamp(max=min_score)

        perm = (x > scores_min).nonzero(as_tuple=False).view(-1)
    else:
        num_nodes = scatter_add(batch.new_ones(x.size(0)), batch, dim=0)
        batch_size, max_num_nodes = num_nodes.size(0), num_nodes.max().item()

        cum_num_nodes = torch.cat(
            [num_nodes.new_zeros(1),
             num_nodes.cumsum(dim=0)[:-1]], dim=0)

        index = torch.arange(batch.size(0), dtype=torch.long, device=x.device)
        index = (index - cum_num_nodes[batch]) + (batch * max_num_nodes)

        dense_x = x.new_full((batch_size * max_num_nodes, ),
                             torch.finfo(x.dtype).min)
        dense_x[index] = x
        dense_x = dense_x.view(batch_size, max_num_nodes)

        _, perm = dense_x.sort(dim=-1, descending=True)

        perm = perm + cum_num_nodes.view(-1, 1)
        perm = perm.view(-1)

        if isinstance(ratio, int):
            k = num_nodes.new_full((num_nodes.size(0), ), ratio)
            k = torch.min(k, num_nodes)
        else:
            k = (ratio * num_nodes.to(torch.float)).ceil().to(torch.long)

        mask = [
            torch.arange(k[i], dtype=torch.long, device=x.device) +
            i * max_num_nodes for i in range(batch_size)
        ]
        mask = torch.cat(mask, dim=0)

        perm = perm[mask]

    return perm


def filter_adj(edge_index, edge_attr, perm, num_nodes=None):
    num_nodes = maybe_num_nodes(edge_index, num_nodes)

    mask = perm.new_full((num_nodes, ), -1)
    i = torch.arange(perm.size(0), dtype=torch.long, device=perm.device)
    mask[perm] = i

    row, col = edge_index
    row, col = mask[row], mask[col]
    mask = (row >= 0) & (col >= 0)
    row, col = row[mask], col[mask]

    if edge_attr is not None:
        edge_attr = edge_attr[mask]

    return torch.stack([row, col], dim=0), edge_attr


class GSAPool(torch.nn.Module):

    def __init__(self, in_channels, pooling_ratio=0.5, alpha=0.6,
                        min_score=None, multiplier=1,
                        non_linearity=torch.tanh,
                        cus_drop_ratio =0):
        super(GSAPool,self).__init__()
        self.in_channels = in_channels

        self.ratio = pooling_ratio
        self.alpha = alpha

        self.sbtl_layer = GCNConv(in_channels,1)
        self.fbtl_layer = nn.Linear(in_channels, 1)
        self.fusion = GCNConv(in_channels,in_channels)

        self.min_score = min_score
        self.multiplier = multiplier
        self.fusion_flag = 0
        self.non_linearity = non_linearity

        self.dropout = torch.nn.Dropout(cus_drop_ratio)

    def conv_selection(self, conv, in_channels, conv_type=0):
        if(conv_type == 0):
            out_channels = 1
        elif(conv_type == 1):
            out_channels = in_channels
        if(conv == "GCNConv"):
            return GCNConv(in_channels,out_channels)
        elif(conv == "ChebConv"):
            return ChebConv(in_channels,out_channels,1)
        elif(conv == "SAGEConv"):
            return SAGEConv(in_channels,out_channels)
        elif(conv == "GATConv"):
            return GATConv(in_channels,out_channels, heads=1, concat=True)
        elif(conv == "GraphConv"):
            return GraphConv(in_channels,out_channels)
        else:
            raise ValueError

    def forward(self, x, edge_index, edge_attr=None, batch=None):
        if batch is None:
            batch = edge_index.new_zeros(x.size(0))
        x = x.unsqueeze(-1) if x.dim() == 1 else x

        #SBTL
        score_s = self.sbtl_layer(x,edge_index).squeeze()
        #FBTL
        score_f = self.fbtl_layer(x).squeeze()
        #hyperparametr alpha
        score = score_s*self.alpha + score_f*(1-self.alpha)

        score = score.unsqueeze(-1) if score.dim()==0 else score

        if self.min_score is None:
            score = self.non_linearity(score)
        else:
            score = softmax(score, batch)

        sc = self.dropout(score)
        perm = topk(sc, self.ratio, batch)

        #fusion
        if(self.fusion_flag == 1):
            x = self.fusion(x, edge_index)
        x_ae = x[perm]
        x = x[perm] * score[perm].view(-1, 1)
        x = self.multiplier * x if self.multiplier != 1 else x

        batch = batch[perm]
        edge_index, edge_attr = filter_adj(
            edge_index, edge_attr, perm, num_nodes=score.size(0))

        return x, edge_index, edge_attr, batch, perm, x_ae

In [15]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
num_classes = dataset_Texas.num_classes
in_channels = dataset_Texas.num_features
hidden_channels = 64
out_channels = num_classes
depth = 2
pool_ratios = [0.7, 0.7]
class HierarchicalGCN_GSA(torch.nn.Module):
    def __init__(self, in_channels, hidden_channels, out_channels, depth, pool_ratios, act=F.relu, sum_res=False):
        super(HierarchicalGCN_GSA, self).__init__()
        assert depth >= 1
        self.in_channels = in_channels
        self.hidden_channels = hidden_channels
        self.out_channels = out_channels
        self.depth = depth
        self.pool_ratios = pool_ratios
        self.act = act
        self.sum_res = sum_res

        channels = self.hidden_channels

        self.down_convs = torch.nn.ModuleList()
        self.pools = torch.nn.ModuleList()
        self.down_convs.append(GCNConv(self.in_channels, channels))
        for i in range(self.depth):
            self.pools.append(GSAPool(64, pooling_ratio=pool_ratios[i], alpha = 0.6, cus_drop_ratio = 0))
            self.down_convs.append(GCNConv(channels, channels))

        in_channels = channels if sum_res else 2 * channels

        self.up_convs = torch.nn.ModuleList()
        for i in range(self.depth):
            self.up_convs.append(GCNConv(in_channels, channels))
        self.up_convs.append(GCNConv(channels, self.out_channels))

    def forward(self, x, edge_index, batch=None):
        x, edge_index = x.to(device), edge_index.to(device)
        if batch is None:
            batch = edge_index.new_zeros(x.size(0))
        if batch is not None:
            batch = batch.to(device)

        x = F.dropout(x, p=0.5, training=self.training)
        x = self.down_convs[0](x, edge_index)
        x = self.act(x)

        xs = [x]
        edge_indices = [edge_index]
        perms = []

        for i in range(1, self.depth + 1):
            x, edge_index, edge_attr, batch, perm, _ = self.pools[i - 1](x, edge_index, None, batch)
            x = self.down_convs[i](x, edge_index)
            x = self.act(x)

            if i < self.depth:
                xs.append(x)
                edge_indices.append(edge_index)
            perms.append(perm)

        for i in range(self.depth):
            j = self.depth - 1 - i

            res = xs[j]
            edge_index = edge_indices[j]
            perm = perms[j]

            up = torch.zeros_like(res)
            up[perm] = x
            x = res + up if self.sum_res else torch.cat((res, up), dim=-1)
            x = self.up_convs[i](x, edge_index)
            x = self.act(x)
        x = self.up_convs[-1](x, edge_index)

        return x

#  
def train_node_classifier(model, graph, optimizer, criterion, train_mask, val_mask, n_epochs=200, patience=150, min_delta=0.0001):
    best_val_acc = 0
    patience_counter = 0
    model.to(device)
    graph = graph.to(device)  

    for epoch in range(1, n_epochs + 1):
        model.train()
        optimizer.zero_grad()
        out = model(graph.x, graph.edge_index)
        loss = criterion(out[train_mask], graph.y[train_mask])
        loss.backward()
        optimizer.step()

        val_acc = eval_node_classifier(model, graph, val_mask)

        # Early stopping logic
        if val_acc > best_val_acc + min_delta:
            best_val_acc = val_acc
            patience_counter = 0  # Reset patience counter if improvement
        else:
            patience_counter += 1  # Increment patience counter if no improvement

        if patience_counter >= patience:
            # Early stopping
            break

    return model, best_val_acc

def eval_node_classifier(model, graph, mask):
    model.eval()
    pred = model(graph.x, graph.edge_index).argmax(dim=1)
    correct = (pred[mask] == graph.y[mask]).sum()
    acc = int(correct) / int(mask.sum())
    return acc

def compute_f1_scores(model, graph, mask):
    model.eval()
    pred = model(graph.x, graph.edge_index).argmax(dim=1)
    y_true = graph.y[mask].cpu().numpy()
    y_pred = pred[mask].cpu().numpy()
    micro_f1 = f1_score(y_true, y_pred, average='micro')
    macro_f1 = f1_score(y_true, y_pred, average='macro')
    return micro_f1, macro_f1


seeds = [42, 123, 456]
micro_f1_scores = []
macro_f1_scores = []

for seed in seeds:
    torch.manual_seed(seed)
    np.random.seed(seed)

    model = HierarchicalGCN_GSA(in_channels=in_channels, hidden_channels=hidden_channels,
                                 out_channels=out_channels, depth=depth, pool_ratios=pool_ratios)
    optimizer = torch.optim.Adam(model.parameters(), lr=0.001, weight_decay=5e-4)
    criterion = torch.nn.CrossEntropyLoss()

    model, best_val_acc = train_node_classifier(model, data, optimizer, criterion, data.train_mask, data.val_mask)

    micro_f1, macro_f1 = compute_f1_scores(model, data, data.val_mask)
    micro_f1_scores.append(micro_f1)
    macro_f1_scores.append(macro_f1)

micro_f1_mean = np.mean(micro_f1_scores)
micro_f1_std = np.std(micro_f1_scores)
macro_f1_mean = np.mean(macro_f1_scores)
macro_f1_std = np.std(macro_f1_scores)

micro_f1_mean = np.mean(micro_f1_scores) * 100
micro_f1_std = np.std(micro_f1_scores) * 100
macro_f1_mean = np.mean(macro_f1_scores) * 100
macro_f1_std = np.std(macro_f1_scores) * 100

print(f"Micro F1: {micro_f1_mean:.2f}$\pm${micro_f1_std:.2f}")
print(f"Macro F1: {macro_f1_mean:.2f}$\pm${macro_f1_std:.2f}")

Micro F1: 62.22$\pm$0.00
Macro F1: 19.44$\pm$0.00


### HGPSLPooling

In [16]:
import torch
import torch.nn as nn
from torch.autograd import Function
from torch_scatter import scatter_add, scatter_max
from torch_geometric.utils import softmax, dense_to_sparse, add_remaining_self_loops
def topk(x, ratio, batch, min_score=None, tol=1e-7):

    if min_score is not None:
        # Make sure that we do not drop all nodes in a graph.
        scores_max = scatter_max(x, batch)[0][batch] - tol
        scores_min = scores_max.clamp(max=min_score)

        perm = torch.nonzero(x > scores_min).view(-1)
    else:
        num_nodes = scatter_add(batch.new_ones(x.size(0)), batch, dim=0)
        batch_size, max_num_nodes = num_nodes.size(0), num_nodes.max().item()

        cum_num_nodes = torch.cat(
            [num_nodes.new_zeros(1),
            num_nodes.cumsum(dim=0)[:-1]], dim=0)

        index = torch.arange(batch.size(0), dtype=torch.long, device=x.device)
        index = (index - cum_num_nodes[batch]) + (batch * max_num_nodes)

        dense_x = x.new_full((batch_size * max_num_nodes, ), -2)
        dense_x[index] = x
        dense_x = dense_x.view(batch_size, max_num_nodes)

        _, perm = dense_x.sort(dim=-1, descending=True)

        perm = perm + cum_num_nodes.view(-1, 1)
        perm = perm.view(-1)

        k = (ratio * num_nodes.to(torch.float)).ceil().to(torch.long)
        mask = [
            torch.arange(k[i], dtype=torch.long, device=x.device) +
            i * max_num_nodes for i in range(batch_size)
        ]
        mask = torch.cat(mask, dim=0)

        perm = perm[mask]

    return perm

def filter_adj(edge_index, edge_weight, perm, num_nodes=None):

        num_nodes = maybe_num_nodes(edge_index, num_nodes)

        mask = perm.new_full((num_nodes, ), -1)
        i = torch.arange(perm.size(0), dtype=torch.long, device=perm.device)
        mask[perm] = i

        row, col = edge_index
        row, col = mask[row], mask[col]
        mask = (row >= 0) & (col >= 0)
        row, col = row[mask], col[mask]

        if edge_weight is not None:
            edge_weight = edge_weight[mask]

        return torch.stack([row, col], dim=0), edge_weight

def scatter_sort(x, batch, fill_value=-1e16):
    num_nodes = scatter_add(batch.new_ones(x.size(0)), batch, dim=0)
    batch_size, max_num_nodes = num_nodes.size(0), num_nodes.max().item()

    cum_num_nodes = torch.cat([num_nodes.new_zeros(1), num_nodes.cumsum(dim=0)[:-1]], dim=0)

    index = torch.arange(batch.size(0), dtype=torch.long, device=x.device)
    index = (index - cum_num_nodes[batch]) + (batch * max_num_nodes)

    dense_x = x.new_full((batch_size * max_num_nodes,), fill_value)
    dense_x[index] = x
    dense_x = dense_x.view(batch_size, max_num_nodes)

    sorted_x, _ = dense_x.sort(dim=-1, descending=True)
    cumsum_sorted_x = sorted_x.cumsum(dim=-1)
    cumsum_sorted_x = cumsum_sorted_x.view(-1)

    sorted_x = sorted_x.view(-1)
    filled_index = sorted_x != fill_value

    sorted_x = sorted_x[filled_index]
    cumsum_sorted_x = cumsum_sorted_x[filled_index]

    return sorted_x, cumsum_sorted_x


def _make_ix_like(batch):
    num_nodes = scatter_add(batch.new_ones(batch.size(0)), batch, dim=0)
    idx = [torch.arange(1, i + 1, dtype=torch.long, device=batch.device) for i in num_nodes]
    idx = torch.cat(idx, dim=0)

    return idx


def _threshold_and_support(x, batch):
    """Sparsemax building block: compute the threshold
    Args:
        x: input tensor to apply the sparsemax
        batch: group indicators
    Returns:
        the threshold value
    """
    num_nodes = scatter_add(batch.new_ones(x.size(0)), batch, dim=0)
    cum_num_nodes = torch.cat([num_nodes.new_zeros(1), num_nodes.cumsum(dim=0)[:-1]], dim=0)

    sorted_input, input_cumsum = scatter_sort(x, batch)
    input_cumsum = input_cumsum - 1.0
    rhos = _make_ix_like(batch).to(x.dtype)
    support = rhos * sorted_input > input_cumsum

    support_size = scatter_add(support.to(batch.dtype), batch)
    # mask invalid index, for example, if batch is not start from 0 or not continuous, it may result in negative index
    idx = support_size + cum_num_nodes - 1
    mask = idx < 0
    idx[mask] = 0
    tau = input_cumsum.gather(0, idx)
    tau /= support_size.to(x.dtype)

    return tau, support_size


class SparsemaxFunction(Function):

    @staticmethod
    def forward(ctx, x, batch):
        """sparsemax: normalizing sparse transform
        Parameters:
            ctx: context object
            x (Tensor): shape (N, )
            batch: group indicator
        Returns:
            output (Tensor): same shape as input
        """
        max_val, _ = scatter_max(x, batch)
        x -= max_val[batch]
        tau, supp_size = _threshold_and_support(x, batch)
        output = torch.clamp(x - tau[batch], min=0)
        ctx.save_for_backward(supp_size, output, batch)

        return output

    @staticmethod
    def backward(ctx, grad_output):
        supp_size, output, batch = ctx.saved_tensors
        grad_input = grad_output.clone()
        grad_input[output == 0] = 0

        v_hat = scatter_add(grad_input, batch) / supp_size.to(output.dtype)
        grad_input = torch.where(output != 0, grad_input - v_hat[batch], grad_input)

        return grad_input, None


sparsemax = SparsemaxFunction.apply


class Sparsemax(nn.Module):

    def __init__(self):
        super(Sparsemax, self).__init__()

    def forward(self, x, batch):
        return sparsemax(x, batch)


if __name__ == '__main__':
    sparse_attention = Sparsemax()
    input_x = torch.tensor([1.7301, 0.6792, -1.0565, 1.6614, -0.3196, -0.7790, -0.3877, -0.4943, 0.1831, -0.0061])
    input_batch = torch.cat([torch.zeros(4, dtype=torch.long), torch.ones(6, dtype=torch.long)], dim=0)
    res = sparse_attention(input_x, input_batch)
    print(res)

class TwoHopNeighborhood(object):
    def __call__(self, data):
        edge_index, edge_attr = data.edge_index, data.edge_attr
        n = data.num_nodes

        fill = 1e16
        value = edge_index.new_full((edge_index.size(1),), fill, dtype=torch.float)

        index, value = spspmm(edge_index, value, edge_index, value, n, n, n, True)

        edge_index = torch.cat([edge_index, index], dim=1)
        if edge_attr is None:
            data.edge_index, _ = coalesce(edge_index, None, n, n)
        else:
            value = value.view(-1, *[1 for _ in range(edge_attr.dim() - 1)])
            value = value.expand(-1, *list(edge_attr.size())[1:])
            edge_attr = torch.cat([edge_attr, value], dim=0)
            data.edge_index, edge_attr = coalesce(edge_index, edge_attr, n, n, op='min', fill_value=fill)
            edge_attr[edge_attr >= fill] = 0
            data.edge_attr = edge_attr

        return data

    def __repr__(self):
        return '{}()'.format(self.__class__.__name__)


class GCN(MessagePassing):
    def __init__(self, in_channels, out_channels, cached=False, bias=True, **kwargs):
        super(GCN, self).__init__(aggr='add', **kwargs)

        self.in_channels = in_channels
        self.out_channels = out_channels
        self.cached = cached
        self.cached_result = None
        self.cached_num_edges = None

        self.weight = Parameter(torch.Tensor(in_channels, out_channels))
        nn.init.xavier_uniform_(self.weight.data)

        if bias:
            self.bias = Parameter(torch.Tensor(out_channels))
            nn.init.zeros_(self.bias.data)
        else:
            self.register_parameter('bias', None)

        self.reset_parameters()

    def reset_parameters(self):
        self.cached_result = None
        self.cached_num_edges = None

    @staticmethod
    def norm(edge_index, num_nodes, edge_weight, dtype=None):
        if edge_weight is None:
            edge_weight = torch.ones((edge_index.size(1),), dtype=dtype, device=edge_index.device)

        row, col = edge_index
        deg = scatter_add(edge_weight, row, dim=0, dim_size=num_nodes)
        deg_inv_sqrt = deg.pow(-0.5)
        deg_inv_sqrt[deg_inv_sqrt == float('inf')] = 0

        return edge_index, deg_inv_sqrt[row] * edge_weight * deg_inv_sqrt[col]

    def forward(self, x, edge_index, edge_weight=None):
        x = torch.matmul(x, self.weight)

        if self.cached and self.cached_result is not None:
            if edge_index.size(1) != self.cached_num_edges:
                raise RuntimeError(
                    'Cached {} number of edges, but found {}'.format(self.cached_num_edges, edge_index.size(1)))

        if not self.cached or self.cached_result is None:
            self.cached_num_edges = edge_index.size(1)
            edge_index, norm = self.norm(edge_index, x.size(0), edge_weight, x.dtype)
            self.cached_result = edge_index, norm

        edge_index, norm = self.cached_result

        return self.propagate(edge_index, x=x, norm=norm)

    def message(self, x_j, norm):
        return norm.view(-1, 1) * x_j

    def update(self, aggr_out):
        if self.bias is not None:
            aggr_out = aggr_out + self.bias
        return aggr_out

    def __repr__(self):
        return '{}({}, {})'.format(self.__class__.__name__, self.in_channels, self.out_channels)


class NodeInformationScore(MessagePassing):
    def __init__(self, improved=False, cached=False, **kwargs):
        super(NodeInformationScore, self).__init__(aggr='add', **kwargs)

        self.improved = improved
        self.cached = cached
        self.cached_result = None
        self.cached_num_edges = None

    @staticmethod
    def norm(edge_index, num_nodes, edge_weight, dtype=None):
        if edge_weight is None:
            edge_weight = torch.ones((edge_index.size(1),), dtype=dtype, device=edge_index.device)

        row, col = edge_index
        deg = scatter_add(edge_weight, row, dim=0, dim_size=num_nodes)
        deg_inv_sqrt = deg.pow(-0.5)
        deg_inv_sqrt[deg_inv_sqrt == float('inf')] = 0

        edge_index, edge_weight = add_remaining_self_loops(edge_index, edge_weight, 0, num_nodes)

        row, col = edge_index
        expand_deg = torch.zeros((edge_weight.size(0),), dtype=dtype, device=edge_index.device)
        expand_deg[-num_nodes:] = torch.ones((num_nodes,), dtype=dtype, device=edge_index.device)

        return edge_index, expand_deg - deg_inv_sqrt[row] * edge_weight * deg_inv_sqrt[col]

    def forward(self, x, edge_index, edge_weight):
        if self.cached and self.cached_result is not None:
            if edge_index.size(1) != self.cached_num_edges:
                raise RuntimeError(
                    'Cached {} number of edges, but found {}'.format(self.cached_num_edges, edge_index.size(1)))

        if not self.cached or self.cached_result is None:
            self.cached_num_edges = edge_index.size(1)
            edge_index, norm = self.norm(edge_index, x.size(0), edge_weight, x.dtype)
            self.cached_result = edge_index, norm

        edge_index, norm = self.cached_result

        return self.propagate(edge_index, x=x, norm=norm)

    def message(self, x_j, norm):
        return norm.view(-1, 1) * x_j

    def update(self, aggr_out):
        return aggr_out


class HGPSLPool(torch.nn.Module):
    def __init__(self, in_channels, ratio=0.5, sample=False, sparse=False, sl=True, lamb=1.0, negative_slop=0.2):
        super(HGPSLPool, self).__init__()
        self.in_channels = in_channels
        self.ratio = ratio
        self.sample = sample
        self.sparse = sparse
        self.sl = sl
        self.negative_slop = negative_slop
        self.lamb = lamb

        self.att = Parameter(torch.Tensor(1, self.in_channels * 2))
        nn.init.xavier_uniform_(self.att.data)
        self.sparse_attention = Sparsemax()
        self.neighbor_augment = TwoHopNeighborhood()
        self.calc_information_score = NodeInformationScore()

    def forward(self, x, edge_index, edge_attr, batch):
        if batch is None:
            batch = edge_index.new_zeros(x.size(0))

        x_information_score = self.calc_information_score(x, edge_index, edge_attr)
        score = torch.sum(torch.abs(x_information_score), dim=1)

        # Graph Pooling
        original_x = x
        perm = topk(score, self.ratio, batch)
        x = x[perm]
        batch = batch[perm]
        induced_edge_index, induced_edge_attr = filter_adj(edge_index, edge_attr, perm, num_nodes=score.size(0))

        # Discard structure learning layer, directly return
        if self.sl is False:
            return x, induced_edge_index, induced_edge_attr, batch

        # Structure Learning
        if self.sample:
            # A fast mode for large graphs.
            # In large graphs, learning the possible edge weights between each pair of nodes is time consuming.
            # To accelerate this process, we sample it's K-Hop neighbors for each node and then learn the
            # edge weights between them.
            k_hop = 3
            if edge_attr is None:
                edge_attr = torch.ones((edge_index.size(1),), dtype=torch.float, device=edge_index.device)

            hop_data = Data(x=original_x, edge_index=edge_index, edge_attr=edge_attr)
            for _ in range(k_hop - 1):
                hop_data = self.neighbor_augment(hop_data)
            hop_edge_index = hop_data.edge_index
            hop_edge_attr = hop_data.edge_attr
            new_edge_index, new_edge_attr = filter_adj(hop_edge_index, hop_edge_attr, perm, num_nodes=score.size(0))

            new_edge_index, new_edge_attr = add_remaining_self_loops(new_edge_index, new_edge_attr, 0, x.size(0))
            row, col = new_edge_index
            weights = (torch.cat([x[row], x[col]], dim=1) * self.att).sum(dim=-1)
            weights = F.leaky_relu(weights, self.negative_slop) + new_edge_attr * self.lamb
            adj = torch.zeros((x.size(0), x.size(0)), dtype=torch.float, device=x.device)
            adj[row, col] = weights
            new_edge_index, weights = dense_to_sparse(adj)
            row, col = new_edge_index
            if self.sparse:
                new_edge_attr = self.sparse_attention(weights, row)
            else:
                new_edge_attr = softmax(weights, row, x.size(0))
            # filter out zero weight edges
            adj[row, col] = new_edge_attr
            new_edge_index, new_edge_attr = dense_to_sparse(adj)
            # release gpu memory
            del adj
            torch.cuda.empty_cache()
        else:
            # Learning the possible edge weights between each pair of nodes in the pooled subgraph, relative slower.
            if edge_attr is None:
                induced_edge_attr = torch.ones((induced_edge_index.size(1),), dtype=x.dtype,
                                               device=induced_edge_index.device)
            num_nodes = scatter_add(batch.new_ones(x.size(0)), batch, dim=0)
            shift_cum_num_nodes = torch.cat([num_nodes.new_zeros(1), num_nodes.cumsum(dim=0)[:-1]], dim=0)
            cum_num_nodes = num_nodes.cumsum(dim=0)
            adj = torch.zeros((x.size(0), x.size(0)), dtype=torch.float, device=x.device)
            # Construct batch fully connected graph in block diagonal matirx format
            for idx_i, idx_j in zip(shift_cum_num_nodes, cum_num_nodes):
                adj[idx_i:idx_j, idx_i:idx_j] = 1.0
            new_edge_index, _ = dense_to_sparse(adj)
            row, col = new_edge_index

            weights = (torch.cat([x[row], x[col]], dim=1) * self.att).sum(dim=-1)
            weights = F.leaky_relu(weights, self.negative_slop)
            adj[row, col] = weights
            induced_row, induced_col = induced_edge_index

            adj[induced_row, induced_col] += induced_edge_attr * self.lamb
            weights = adj[row, col]
            if self.sparse:
                new_edge_attr = self.sparse_attention(weights, row)
            else:
                new_edge_attr = softmax(weights, row, num_nodes=x.size(0))
            # filter out zero weight edges
            adj[row, col] = new_edge_attr
            new_edge_index, new_edge_attr = dense_to_sparse(adj)
            # release gpu memory
            del adj
            torch.cuda.empty_cache()

        return x, new_edge_index, new_edge_attr, batch, perm

tensor([0.5344, 0.0000, 0.0000, 0.4656, 0.0613, 0.0000, 0.0000, 0.0000, 0.5640,
        0.3748])


In [17]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
num_classes = dataset_Texas.num_classes
in_channels = dataset_Texas.num_features
hidden_channels = 64
out_channels = num_classes
depth = 2
pool_ratios = [0.7, 0.7]
class HierarchicalGCN_HGPSL(torch.nn.Module):
    def __init__(self, in_channels, hidden_channels, out_channels, depth, pool_ratios, act=F.relu, sum_res=False):
        super(HierarchicalGCN_HGPSL, self).__init__()
        assert depth >= 1
        self.in_channels = in_channels
        self.hidden_channels = hidden_channels
        self.out_channels = out_channels
        self.depth = depth
        self.pool_ratios = pool_ratios
        self.act = act
        self.sum_res = sum_res

        channels = self.hidden_channels

        self.down_convs = torch.nn.ModuleList()
        self.pools = torch.nn.ModuleList()
        self.down_convs.append(GCNConv(self.in_channels, channels))
        for i in range(self.depth):
            self.pools.append(HGPSLPool(hidden_channels, ratio=pool_ratios[i], sample=False, sparse=False, sl=True, lamb=1.0, negative_slop=0.2))
            self.down_convs.append(GCNConv(channels, channels))

        in_channels = channels if sum_res else 2 * channels

        self.up_convs = torch.nn.ModuleList()
        for i in range(self.depth):
            self.up_convs.append(GCNConv(in_channels, channels))
        self.up_convs.append(GCNConv(channels, self.out_channels))

    def forward(self, x, edge_index, batch=None):
        x, edge_index = x.to(device), edge_index.to(device)
        edge_attr = None
        if batch is None:
            batch = edge_index.new_zeros(x.size(0))
        if batch is not None:
            batch = batch.to(device)

        x = F.dropout(x, p=0.5, training=self.training)
        x = self.down_convs[0](x, edge_index)
        x = self.act(x)

        xs = [x]
        edge_indices = [edge_index]
        perms = []

        for i in range(1, self.depth + 1):
            x, edge_index, _, batch, perm = self.pools[i - 1](x, edge_index, edge_attr, batch)
            x = self.down_convs[i](x, edge_index)
            x = self.act(x)

            if i < self.depth:
                xs.append(x)
                edge_indices.append(edge_index)
            perms.append(perm)

        for i in range(self.depth):
            j = self.depth - 1 - i

            res = xs[j]
            edge_index = edge_indices[j]
            perm = perms[j]

            up = torch.zeros_like(res)
            up[perm] = x
            x = res + up if self.sum_res else torch.cat((res, up), dim=-1)
            x = self.up_convs[i](x, edge_index)
            x = self.act(x)
        x = self.up_convs[-1](x, edge_index)

        return x

def train_node_classifier(model, graph, optimizer, criterion, train_mask, val_mask, n_epochs=200, patience=150, min_delta=0.0001):
    best_val_acc = 0
    patience_counter = 0
    model.to(device)
    graph = graph.to(device)  

    for epoch in range(1, n_epochs + 1):
        model.train()
        optimizer.zero_grad()
        out = model(graph.x, graph.edge_index)
        loss = criterion(out[train_mask], graph.y[train_mask])
        loss.backward()
        optimizer.step()

        # Evaluate on validation set
        val_acc = eval_node_classifier(model, graph, val_mask)
        micro_f1, macro_f1 = compute_f1_scores(model, graph, val_mask)

        # Early stopping logic
        if val_acc > best_val_acc + min_delta:
            best_val_acc = val_acc
            patience_counter = 0  # Reset patience counter if improvement
        else:
            patience_counter += 1  # Increment patience counter if no improvement

        if patience_counter >= patience:
            # Early stopping
            print(f'Early stopping at epoch {epoch}')
            break

        # Print epoch progress
        print(f'Epoch {epoch:03d}: Loss = {loss.item():.4f}, Val Acc = {val_acc:.4f}, Micro F1 = {micro_f1:.4f}, Macro F1 = {macro_f1:.4f}')

    return model, best_val_acc

# Example usage with the loop over seeds
seeds = [42, 123, 456]
micro_f1_scores = []
macro_f1_scores = []

for seed in seeds:
    torch.manual_seed(seed)
    np.random.seed(seed)

    model = HierarchicalGCN_HGPSL(in_channels=in_channels, hidden_channels=hidden_channels,
                                 out_channels=out_channels, depth=depth, pool_ratios=pool_ratios)
    optimizer = torch.optim.Adam(model.parameters(), lr=0.001, weight_decay=5e-4)
    criterion = torch.nn.CrossEntropyLoss()

    # Train the model
    model, best_val_acc = train_node_classifier(model, data, optimizer, criterion, data.train_mask, data.val_mask)

    # Compute F1 scores
    micro_f1, macro_f1 = compute_f1_scores(model, data, data.val_mask)
    micro_f1_scores.append(micro_f1)
    macro_f1_scores.append(macro_f1)

# Calculate mean and std deviation of F1 scores
micro_f1_mean = np.mean(micro_f1_scores)
micro_f1_std = np.std(micro_f1_scores)
macro_f1_mean = np.mean(macro_f1_scores)
macro_f1_std = np.std(macro_f1_scores)

micro_f1_mean = np.mean(micro_f1_scores) * 100
micro_f1_std = np.std(micro_f1_scores) * 100
macro_f1_mean = np.mean(macro_f1_scores) * 100
macro_f1_std = np.std(macro_f1_scores) * 100

# 打印输出格式化后的结果
print(f"Micro F1: {micro_f1_mean:.2f}$\pm${micro_f1_std:.2f}")
print(f"Macro F1: {macro_f1_mean:.2f}$\pm${macro_f1_std:.2f}")


Epoch 001: Loss = 1.6338, Val Acc = 0.0222, Micro F1 = 0.0222, Macro F1 = 0.0200
Epoch 002: Loss = 1.5577, Val Acc = 0.0222, Micro F1 = 0.0222, Macro F1 = 0.1667
Epoch 003: Loss = 1.4985, Val Acc = 0.0222, Micro F1 = 0.0222, Macro F1 = 0.0500
Epoch 004: Loss = 1.4271, Val Acc = 0.0222, Micro F1 = 0.0222, Macro F1 = 0.0417
Epoch 005: Loss = 1.4213, Val Acc = 0.0222, Micro F1 = 0.0222, Macro F1 = 0.0152
Epoch 006: Loss = 1.3563, Val Acc = 0.0222, Micro F1 = 0.0222, Macro F1 = 0.0114
Epoch 007: Loss = 1.3331, Val Acc = 0.0222, Micro F1 = 0.0222, Macro F1 = 0.0111
Epoch 008: Loss = 1.2809, Val Acc = 0.4222, Micro F1 = 0.4222, Macro F1 = 0.1661
Epoch 009: Loss = 1.2722, Val Acc = 0.6222, Micro F1 = 0.6222, Macro F1 = 0.2196
Epoch 010: Loss = 1.2384, Val Acc = 0.7556, Micro F1 = 0.7556, Macro F1 = 0.2643
Epoch 011: Loss = 1.2052, Val Acc = 0.7778, Micro F1 = 0.7778, Macro F1 = 0.2188
Epoch 012: Loss = 1.1769, Val Acc = 0.8667, Micro F1 = 0.8667, Macro F1 = 0.3095
Epoch 013: Loss = 1.1383, Va